In [1]:
# Table Tennis (Pong-style) with PNGs from your Downloads folder
# -------------------------------------------------------------
# Features
# - Loads paddle and ball PNGs from your Downloads folder (Windows/Mac/Linux)
# - If images are missing, it will try to DOWNLOAD sample images into Downloads
#   and load them automatically. If download fails (no internet), it will draw
#   clean vector shapes and also SAVE placeholder PNGs into Downloads for you.
# - 1 Player vs Computer AI (adjustable difficulty)
# - Smooth physics: paddle spin (angle changes based on impact point)
# - Pause (P), Restart (R), Quit (ESC)
# - Works in 16:9 window, scalable assets
#
# Controls
# - Up/Down arrows  OR  W/S to move
# - P to pause, R to restart, ESC to quit
#
# Notes
# - You can replace the auto-downloaded PNGs later with your own
#   (same filenames) in your Downloads folder and they will be picked up.
# - Filenames used in Downloads:  paddle.png, ball.png, table.png

import os
import sys
import math
import random
from pathlib import Path

# ---- Resolve user's Downloads path cross‑platform ----
HOME = Path.home()
DOWNLOADS = HOME / "Downloads"
DOWNLOADS.mkdir(parents=True, exist_ok=True)

# Asset filenames
PADDLE_PNG = DOWNLOADS / "paddle.png"
BALL_PNG = DOWNLOADS / "ball.png"
TABLE_PNG = DOWNLOADS / "table.png"

# Sample asset URLs (public domain / CC0 style or generic placeholders)
SAMPLE_PADDLE_URL = "https://raw.githubusercontent.com/olivercameron/placeholder-assets/main/pong/paddle.png"
SAMPLE_BALL_URL = "https://raw.githubusercontent.com/olivercameron/placeholder-assets/main/pong/ball.png"
SAMPLE_TABLE_URL = "https://raw.githubusercontent.com/olivercameron/placeholder-assets/main/pong/table.png"

# If those URLs ever break, the code will fallback to generated placeholders.

import pygame
import urllib.request

pygame.init()

# -------------------- Settings --------------------
WIDTH, HEIGHT = 960, 540  # 16:9 window
FPS = 120
MARGIN = 20

PADDLE_W, PADDLE_H = 18, 110
BALL_SIZE = 18

PLAYER_SPEED = 520  # pixels/second
CPU_SPEED = 460      # AI paddle speed
CPU_REACTION = 0.22  # [0..1], higher = stronger AI tracking
INITIAL_BALL_SPEED = 420
MAX_BALL_SPEED = 880
SPIN_FACTOR = 6.0    # how much vertical offset changes ball dy
SCORE_TO_WIN = 7

FONT = pygame.font.SysFont("arial", 28)
BIGFONT = pygame.font.SysFont("arial", 56, bold=True)

# ---------------- Asset helpers -------------------
def try_download(url: str, dest: Path) -> bool:
    try:
        with urllib.request.urlopen(url, timeout=8) as r:
            data = r.read()
        dest.write_bytes(data)
        print(f"Downloaded: {dest}")
        return True
    except Exception as e:
        print(f"Download failed for {url}: {e}")
        return False


def ensure_image(path: Path, fallback_url: str, make_placeholder_surface):
    """Ensure an image exists at path. Try to read existing; else download; else generate placeholder.
    Returns (surface, was_loaded_from_file: bool)
    """
    if path.exists():
        try:
            surf = pygame.image.load(str(path)).convert_alpha()
            return surf, True
        except Exception as e:
            print(f"Failed to load existing {path}: {e}")

    # Try download
    if fallback_url and try_download(fallback_url, path):
        try:
            surf = pygame.image.load(str(path)).convert_alpha()
            return surf, True
        except Exception as e:
            print(f"Failed to load downloaded {path}: {e}")

    # Generate placeholder
    surf = make_placeholder_surface()
    try:
        pygame.image.save(surf, str(path))
        print(f"Saved placeholder PNG to {path}")
    except Exception as e:
        print(f"Failed saving placeholder to {path}: {e}")
    return surf, False


# --------------- Placeholder generators ---------------
def make_paddle_surface():
    surf = pygame.Surface((PADDLE_W, PADDLE_H), pygame.SRCALPHA)
    pygame.draw.rect(surf, (230, 230, 230), (0, 0, PADDLE_W, PADDLE_H), border_radius=8)
    pygame.draw.rect(surf, (255, 255, 255), (2, 2, PADDLE_W-4, PADDLE_H-4), border_radius=8)
    return surf


def make_ball_surface():
    surf = pygame.Surface((BALL_SIZE, BALL_SIZE), pygame.SRCALPHA)
    pygame.draw.circle(surf, (255, 255, 255), (BALL_SIZE//2, BALL_SIZE//2), BALL_SIZE//2)
    pygame.draw.circle(surf, (220, 220, 220), (BALL_SIZE//2 + 2, BALL_SIZE//2 + 2), BALL_SIZE//3)
    return surf


def make_table_surface():
    surf = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
    surf.fill((19, 90, 43))  # green table
    # Center line
    pygame.draw.line(surf, (240, 240, 240), (WIDTH//2, 0), (WIDTH//2, HEIGHT), 2)
    # Border
    pygame.draw.rect(surf, (240, 240, 240), (8, 8, WIDTH-16, HEIGHT-16), 4, border_radius=10)
    return surf


# --------------- Load assets (from Downloads) ---------------
TABLE_SURF, _ = ensure_image(TABLE_PNG, SAMPLE_TABLE_URL, make_table_surface)
PADDLE_SURF, _ = ensure_image(PADDLE_PNG, SAMPLE_PADDLE_URL, make_paddle_surface)
BALL_SURF, _ = ensure_image(BALL_PNG, SAMPLE_BALL_URL, make_ball_surface)

# Scale to target sizes (if images are different)
PADDLE_SURF = pygame.transform.smoothscale(PADDLE_SURF, (PADDLE_W, PADDLE_H))
BALL_SURF = pygame.transform.smoothscale(BALL_SURF, (BALL_SIZE, BALL_SIZE))
TABLE_SURF = pygame.transform.smoothscale(TABLE_SURF, (WIDTH, HEIGHT))

# -------------------- Game objects --------------------
class Paddle:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, PADDLE_W, PADDLE_H)
        self.vy = 0.0

    def update(self, dt):
        self.rect.y += int(self.vy * dt)
        if self.rect.top < MARGIN:
            self.rect.top = MARGIN
        if self.rect.bottom > HEIGHT - MARGIN:
            self.rect.bottom = HEIGHT - MARGIN

    def draw(self, screen):
        screen.blit(PADDLE_SURF, self.rect)


class Ball:
    def __init__(self):
        self.rect = pygame.Rect(0, 0, BALL_SIZE, BALL_SIZE)
        self.reset(direction=random.choice([-1, 1]))

    def reset(self, direction=1):
        self.rect.center = (WIDTH//2, HEIGHT//2)
        angle = random.uniform(-0.35, 0.35)  # slight vertical component
        speed = INITIAL_BALL_SPEED
        self.vx = direction * speed * math.cos(angle)
        self.vy = speed * math.sin(angle)

    def update(self, dt):
        self.rect.x += int(self.vx * dt)
        self.rect.y += int(self.vy * dt)

        # Top/bottom walls
        if self.rect.top <= MARGIN:
            self.rect.top = MARGIN
            self.vy = -self.vy
        if self.rect.bottom >= HEIGHT - MARGIN:
            self.rect.bottom = HEIGHT - MARGIN
            self.vy = -self.vy

    def draw(self, screen):
        screen.blit(BALL_SURF, self.rect)


# -------------------- AI logic --------------------
def cpu_ai(cpu: Paddle, ball: Ball, dt: float):
    # Predict target y with some smoothing and reaction delay
    target = ball.rect.centery
    delta = target - cpu.rect.centery
    cpu.vy = max(-CPU_SPEED, min(CPU_SPEED, delta * CPU_REACTION))
    cpu.update(dt)


# -------------------- Utility --------------------
def clamp_ball_speed(ball: Ball):
    speed = math.hypot(ball.vx, ball.vy)
    if speed > MAX_BALL_SPEED:
        scale = MAX_BALL_SPEED / speed
        ball.vx *= scale
        ball.vy *= scale


def collide_ball_with_paddle(ball: Ball, paddle: Paddle, is_left: bool):
    if ball.rect.colliderect(paddle.rect):
        # Move ball out to avoid sticking
        if is_left:
            ball.rect.left = paddle.rect.right
        else:
            ball.rect.right = paddle.rect.left
        # Compute impact offset (-1 top .. +1 bottom)
        offset = (ball.rect.centery - paddle.rect.centery) / (PADDLE_H / 2)
        ball.vx = -ball.vx
        # Add spin: change vy based on where it hit
        ball.vy += offset * SPIN_FACTOR * 40
        # Add a tiny speed-up each hit
        ball.vx *= 1.05
        ball.vy *= 1.05
        clamp_ball_speed(ball)


# -------------------- Game loop --------------------
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Table Tennis — Downloads PNG Edition")
clock = pygame.time.Clock()

# Entities
player = Paddle(MARGIN + 10, HEIGHT//2 - PADDLE_H//2)
cpu = Paddle(WIDTH - MARGIN - 10 - PADDLE_W, HEIGHT//2 - PADDLE_H//2)
ball = Ball()

score_p, score_c = 0, 0
paused = False


def draw_hud():
    # Table/background
    screen.blit(TABLE_SURF, (0, 0))

    # Scores
    s_text = FONT.render(f"You {score_p} : {score_c} CPU", True, (255, 255, 255))
    screen.blit(s_text, (WIDTH//2 - s_text.get_width()//2, 12))

    # Hints
    hint = FONT.render("Up/Down or W/S — P Pause — R Restart — ESC Quit", True, (230, 230, 230))
    screen.blit(hint, (WIDTH//2 - hint.get_width()//2, HEIGHT - 32))


running = True
while running:
    dt = clock.tick(FPS) / 1000.0

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            elif event.key == pygame.K_p:
                paused = not paused
            elif event.key == pygame.K_r:
                # Restart
                score_p = 0
                score_c = 0
                ball.reset(direction=random.choice([-1, 1]))

    # Input
    keys = pygame.key.get_pressed()
    move_up = keys[pygame.K_UP] or keys[pygame.K_w]
    move_down = keys[pygame.K_DOWN] or keys[pygame.K_s]

    if not paused:
        player.vy = 0
        if move_up:
            player.vy = -PLAYER_SPEED
        elif move_down:
            player.vy = PLAYER_SPEED

        player.update(dt)
        cpu_ai(cpu, ball, dt)
        ball.update(dt)

        # Paddle collisions
        collide_ball_with_paddle(ball, player, is_left=True)
        collide_ball_with_paddle(ball, cpu, is_left=False)

        # Scoring (left/right walls)
        if ball.rect.left <= 0:
            score_c += 1
            ball.reset(direction=1)
        elif ball.rect.right >= WIDTH:
            score_p += 1
            ball.reset(direction=-1)

    # Draw
    draw_hud()
    player.draw(screen)
    cpu.draw(screen)
    ball.draw(screen)

    # Win banner
    if score_p >= SCORE_TO_WIN or score_c >= SCORE_TO_WIN:
        winner = "You" if score_p > score_c else "CPU"
        banner = BIGFONT.render(f"{winner} win! Press R to play again.", True, (255, 255, 255))
        screen.blit(banner, (WIDTH//2 - banner.get_width()//2, HEIGHT//2 - banner.get_height()//2))
        paused = True

    pygame.display.flip()

pygame.quit()

# ---------------- How to use your own PNGs ----------------
# 1) Put your custom images in your Downloads folder with these exact names:
#       paddle.png   (recommended size ~18x110, any will be scaled)
#       ball.png     (recommended size ~18x18, any will be scaled)
#       table.png    (background, any 16:9 image will be scaled to window)
# 2) Run this script again — it will load your images from Downloads automatically.


pygame 2.6.1 (SDL 2.28.4, Python 3.12.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
Download failed for https://raw.githubusercontent.com/olivercameron/placeholder-assets/main/pong/table.png: HTTP Error 404: Not Found
Saved placeholder PNG to C:\Users\gagan\Downloads\table.png
Download failed for https://raw.githubusercontent.com/olivercameron/placeholder-assets/main/pong/paddle.png: HTTP Error 404: Not Found
Saved placeholder PNG to C:\Users\gagan\Downloads\paddle.png
Download failed for https://raw.githubusercontent.com/olivercameron/placeholder-assets/main/pong/ball.png: HTTP Error 404: Not Found
Saved placeholder PNG to C:\Users\gagan\Downloads\ball.png


In [2]:
# Table Tennis (Pong-style) with Real Images
# -------------------------------------------------------------
# This version uses REAL images (table, racket, ball) from your Downloads folder.
# Required PNGs:
#   - table.png   → real table tennis table
#   - paddle.png  → real racket
#   - ball.png    → real ping pong ball
#
# Place them in your Downloads folder before running.
# If any image is missing, placeholders will be generated.

import os
import sys
import math
import random
from pathlib import Path

import pygame
import urllib.request

pygame.init()

# ---- Paths ----
HOME = Path.home()
DOWNLOADS = HOME / "Downloads"
DOWNLOADS.mkdir(parents=True, exist_ok=True)

PADDLE_PNG = DOWNLOADS / "paddle.png"
BALL_PNG = DOWNLOADS / "ball.png"
TABLE_PNG = DOWNLOADS / "table.png"

WIDTH, HEIGHT = 960, 540
FPS = 120
MARGIN = 20

PADDLE_W, PADDLE_H = 70, 140   # racket size (scaled)
BALL_SIZE = 28                 # ball size (scaled)

PLAYER_SPEED = 520
CPU_SPEED = 460
CPU_REACTION = 0.22
INITIAL_BALL_SPEED = 420
MAX_BALL_SPEED = 880
SPIN_FACTOR = 6.0
SCORE_TO_WIN = 7

FONT = pygame.font.SysFont("arial", 28)
BIGFONT = pygame.font.SysFont("arial", 56, bold=True)

# ---------------- Asset loader ----------------
def ensure_image(path: Path, make_placeholder_surface):
    if path.exists():
        try:
            return pygame.image.load(str(path)).convert_alpha()
        except:
            pass
    surf = make_placeholder_surface()
    return surf


def make_paddle_surface():
    surf = pygame.Surface((PADDLE_W, PADDLE_H), pygame.SRCALPHA)
    pygame.draw.rect(surf, (200, 0, 0), (0, 0, PADDLE_W, PADDLE_H), border_radius=12)
    return surf


def make_ball_surface():
    surf = pygame.Surface((BALL_SIZE, BALL_SIZE), pygame.SRCALPHA)
    pygame.draw.circle(surf, (255, 255, 255), (BALL_SIZE//2, BALL_SIZE//2), BALL_SIZE//2)
    return surf


def make_table_surface():
    surf = pygame.Surface((WIDTH, HEIGHT))
    surf.fill((19, 90, 43))
    return surf

TABLE_SURF = ensure_image(TABLE_PNG, make_table_surface)
PADDLE_SURF = ensure_image(PADDLE_PNG, make_paddle_surface)
BALL_SURF = ensure_image(BALL_PNG, make_ball_surface)

PADDLE_SURF = pygame.transform.smoothscale(PADDLE_SURF, (PADDLE_W, PADDLE_H))
BALL_SURF = pygame.transform.smoothscale(BALL_SURF, (BALL_SIZE, BALL_SIZE))
TABLE_SURF = pygame.transform.smoothscale(TABLE_SURF, (WIDTH, HEIGHT))

# ---------------- Game objects ----------------
class Paddle:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, PADDLE_W, PADDLE_H)
        self.vy = 0.0

    def update(self, dt):
        self.rect.y += int(self.vy * dt)
        if self.rect.top < MARGIN:
            self.rect.top = MARGIN
        if self.rect.bottom > HEIGHT - MARGIN:
            self.rect.bottom = HEIGHT - MARGIN

    def draw(self, screen):
        screen.blit(PADDLE_SURF, self.rect)


class Ball:
    def __init__(self):
        self.rect = pygame.Rect(0, 0, BALL_SIZE, BALL_SIZE)
        self.reset(direction=random.choice([-1, 1]))

    def reset(self, direction=1):
        self.rect.center = (WIDTH//2, HEIGHT//2)
        angle = random.uniform(-0.35, 0.35)
        speed = INITIAL_BALL_SPEED
        self.vx = direction * speed * math.cos(angle)
        self.vy = speed * math.sin(angle)

    def update(self, dt):
        self.rect.x += int(self.vx * dt)
        self.rect.y += int(self.vy * dt)
        if self.rect.top <= MARGIN:
            self.rect.top = MARGIN
            self.vy = -self.vy
        if self.rect.bottom >= HEIGHT - MARGIN:
            self.rect.bottom = HEIGHT - MARGIN
            self.vy = -self.vy

    def draw(self, screen):
        screen.blit(BALL_SURF, self.rect)


def cpu_ai(cpu: Paddle, ball: Ball, dt: float):
    target = ball.rect.centery
    delta = target - cpu.rect.centery
    cpu.vy = max(-CPU_SPEED, min(CPU_SPEED, delta * CPU_REACTION))
    cpu.update(dt)


def clamp_ball_speed(ball: Ball):
    speed = math.hypot(ball.vx, ball.vy)
    if speed > MAX_BALL_SPEED:
        scale = MAX_BALL_SPEED / speed
        ball.vx *= scale
        ball.vy *= scale


def collide_ball_with_paddle(ball: Ball, paddle: Paddle, is_left: bool):
    if ball.rect.colliderect(paddle.rect):
        if is_left:
            ball.rect.left = paddle.rect.right
        else:
            ball.rect.right = paddle.rect.left
        offset = (ball.rect.centery - paddle.rect.centery) / (PADDLE_H / 2)
        ball.vx = -ball.vx
        ball.vy += offset * SPIN_FACTOR * 40
        ball.vx *= 1.05
        ball.vy *= 1.05
        clamp_ball_speed(ball)


# ---------------- Game loop ----------------
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Table Tennis — Real Images Edition")
clock = pygame.time.Clock()

player = Paddle(MARGIN + 10, HEIGHT//2 - PADDLE_H//2)
cpu = Paddle(WIDTH - MARGIN - 10 - PADDLE_W, HEIGHT//2 - PADDLE_H//2)
ball = Ball()

score_p, score_c = 0, 0
paused = False


def draw_hud():
    screen.blit(TABLE_SURF, (0, 0))
    s_text = FONT.render(f"You {score_p} : {score_c} CPU", True, (255, 255, 255))
    screen.blit(s_text, (WIDTH//2 - s_text.get_width()//2, 12))


running = True
while running:
    dt = clock.tick(FPS) / 1000.0
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            elif event.key == pygame.K_p:
                paused = not paused
            elif event.key == pygame.K_r:
                score_p = 0
                score_c = 0
                ball.reset(direction=random.choice([-1, 1]))

    keys = pygame.key.get_pressed()
    move_up = keys[pygame.K_UP] or keys[pygame.K_w]
    move_down = keys[pygame.K_DOWN] or keys[pygame.K_s]

    if not paused:
        player.vy = 0
        if move_up:
            player.vy = -PLAYER_SPEED
        elif move_down:
            player.vy = PLAYER_SPEED
        player.update(dt)
        cpu_ai(cpu, ball, dt)
        ball.update(dt)
        collide_ball_with_paddle(ball, player, True)
        collide_ball_with_paddle(ball, cpu, False)
        if ball.rect.left <= 0:
            score_c += 1
            ball.reset(direction=1)
        elif ball.rect.right >= WIDTH:
            score_p += 1
            ball.reset(direction=-1)

    draw_hud()
    player.draw(screen)
    cpu.draw(screen)
    ball.draw(screen)

    if score_p >= SCORE_TO_WIN or score_c >= SCORE_TO_WIN:
        winner = "You" if score_p > score_c else "CPU"
        banner = BIGFONT.render(f"{winner} win! Press R to play again.", True, (255, 255, 255))
        screen.blit(banner, (WIDTH//2 - banner.get_width()//2, HEIGHT//2 - banner.get_height()//2))
        paused = True

    pygame.display.flip()

pygame.quit()


In [3]:
"""
Table Tennis (Ping-Pong) game using pygame.
Tries to load real images from ~/Downloads, falls back to drawn shapes if not found.

Controls:
  - Player: A / D or Left / Right to move paddle
  - Space: Serve / Start
  - R: Restart game (reset scores)
  - Esc or window close: Quit

Requirements: pygame
Install: pip install pygame
"""

import os
import sys
import random
import math
import pygame

# SETTINGS
SCREEN_W, SCREEN_H = 1000, 600
FPS = 60
PADDLE_WIDTH = 140
PADDLE_HEIGHT = 24
PADDLE_Y_OFFSET = 30  # distance from bottom/top
BALL_RADIUS = 12
BALL_SPEED_INIT = 6.0
MAX_SCORE = 11
DOWNLOADS = os.path.expanduser("~/Downloads")

# Image filenames to look for in Downloads
FILES = {
    "table": "table.png",
    "player": "player.png",
    "opponent": "opponent.png",
    "racket": "racket.png",
    "ball": "ball.png"
}


def load_image_from_downloads(name, fallback_size=None, convert_alpha=True):
    """
    Attempt to load an image from Downloads by filename in FILES[name].
    Return surface or None if not found.
    If fallback_size provided, scale to that size.
    """
    filename = FILES.get(name)
    if not filename:
        return None
    path = os.path.join(DOWNLOADS, filename)
    if not os.path.isfile(path):
        return None
    try:
        img = pygame.image.load(path)
        if convert_alpha:
            img = img.convert_alpha()
        else:
            img = img.convert()
        if fallback_size:
            img = pygame.transform.smoothscale(img, fallback_size)
        return img
    except Exception as e:
        print(f"Failed to load {path}: {e}")
        return None


class Paddle:
    def __init__(self, x, y, width, height, image=None):
        self.x = x
        self.y = y
        self.w = width
        self.h = height
        self.speed = 9.0
        self.image = image

    def rect(self):
        return pygame.Rect(int(self.x - self.w // 2), int(self.y - self.h // 2), self.w, self.h)

    def draw(self, surface):
        if self.image:
            img = pygame.transform.smoothscale(self.image, (self.w, self.h))
            surface.blit(img, img.get_rect(center=(self.x, self.y)))
        else:
            pygame.draw.rect(surface, (200, 200, 200), self.rect(), border_radius=8)

    def move_left(self):
        self.x -= self.speed
        if self.x - self.w // 2 < 0:
            self.x = self.w // 2

    def move_right(self):
        self.x += self.speed
        if self.x + self.w // 2 > SCREEN_W:
            self.x = SCREEN_W - self.w // 2


class Ball:
    def __init__(self, x, y, radius, image=None):
        self.x = x
        self.y = y
        self.r = radius
        self.image = image
        self.vx = 0.0
        self.vy = 0.0
        self.speed = BALL_SPEED_INIT

    def reset(self, serve_from_top=False):
        # start near the player or opponent depending on serve_from_top
        self.x = SCREEN_W // 2
        self.y = SCREEN_H // 2
        angle = random.uniform(-0.25 * math.pi, 0.25 * math.pi)
        if serve_from_top:
            # ball goes downwards initially
            self.vx = self.speed * math.sin(angle)
            self.vy = abs(self.speed * math.cos(angle))
        else:
            # ball goes upwards initially
            self.vx = self.speed * math.sin(angle)
            self.vy = -abs(self.speed * math.cos(angle))

    def update(self):
        self.x += self.vx
        self.y += self.vy

        # wall collisions left/right
        if self.x - self.r <= 0:
            self.x = self.r
            self.vx *= -1
        if self.x + self.r >= SCREEN_W:
            self.x = SCREEN_W - self.r
            self.vx *= -1

    def draw(self, surface):
        if self.image:
            size = (self.r * 2, self.r * 2)
            img = pygame.transform.smoothscale(self.image, size)
            surface.blit(img, img.get_rect(center=(int(self.x), int(self.y))))
        else:
            pygame.draw.circle(surface, (255, 255, 255), (int(self.x), int(self.y)), self.r)


def paddle_ball_collision(ball: Ball, paddle: Paddle, from_top=False):
    # Only process if ball is moving toward the paddle
    rect = paddle.rect()
    # approximate circle-rect collision
    nearest_x = max(rect.left, min(ball.x, rect.right))
    nearest_y = max(rect.top, min(ball.y, rect.bottom))
    dx = ball.x - nearest_x
    dy = ball.y - nearest_y
    if dx * dx + dy * dy <= ball.r * ball.r:
        # collision happened
        # reflect vertically
        if from_top and ball.vy > 0:
            ball.y = rect.bottom + ball.r
            ball.vy *= -1
        elif not from_top and ball.vy < 0:
            ball.y = rect.top - ball.r
            ball.vy *= -1
        else:
            # if moving other direction, still reflect
            ball.vy *= -1

        # add horizontal velocity based on where it hit the paddle
        offset = (ball.x - paddle.x) / (paddle.w / 2)  # -1 .. 1
        ball.vx += offset * 2.5
        # cap speed
        sp = math.hypot(ball.vx, ball.vy)
        max_sp = 12.0
        if sp > max_sp:
            scale = max_sp / sp
            ball.vx *= scale
            ball.vy *= scale

        # small "spin" or speed boost for more lively gameplay
        ball.vx += offset * 0.6
        # ensure it moves away
        if not from_top and ball.vy > 0:
            ball.vy = -abs(ball.vy)
        if from_top and ball.vy < 0:
            ball.vy = abs(ball.vy)


def draw_text_center(surface, text, size, y, color=(245, 245, 245)):
    font = pygame.font.SysFont("dejavusans", size, bold=True)
    surf = font.render(text, True, color)
    rect = surf.get_rect(center=(SCREEN_W // 2, y))
    surface.blit(surf, rect)


def main():
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_W, SCREEN_H))
    pygame.display.set_caption("Table Tennis - Real Images (fallback supported)")
    clock = pygame.time.Clock()

    # Try to load images
    table_img = load_image_from_downloads("table", fallback_size=(SCREEN_W, SCREEN_H))  # background
    player_img = load_image_from_downloads("player", fallback_size=(PADDLE_WIDTH, PADDLE_HEIGHT))
    opponent_img = load_image_from_downloads("opponent", fallback_size=(PADDLE_WIDTH, PADDLE_HEIGHT))
    ball_img = load_image_from_downloads("ball", fallback_size=(BALL_RADIUS*2, BALL_RADIUS*2))
    racket_img = load_image_from_downloads("racket", fallback_size=(int(PADDLE_WIDTH*0.6), PADDLE_HEIGHT))

    # create paddles & ball
    player = Paddle(SCREEN_W // 2, SCREEN_H - PADDLE_Y_OFFSET, PADDLE_WIDTH, PADDLE_HEIGHT, image=player_img or racket_img)
    opponent = Paddle(SCREEN_W // 2, PADDLE_Y_OFFSET, PADDLE_WIDTH, PADDLE_HEIGHT, image=opponent_img or racket_img)
    ball = Ball(SCREEN_W // 2, SCREEN_H // 2, BALL_RADIUS, image=ball_img)

    # game state
    player_score = 0
    opponent_score = 0
    running = True
    paused = True  # start paused, press space to serve
    serve_from_top = False  # which side serves next (flip on each rally)
    ball.reset(serve_from_top=serve_from_top)
    ai_enabled = True  # opponent controlled by AI
    # simple AI difficulty: how fast it tracks the ball (0..1)
    ai_reaction = 0.16

    # For nicer visuals: table lines
    table_line_color = (40, 120, 40)

    while running:
        dt = clock.tick(FPS) / 1000.0

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_SPACE:
                    # serve / start
                    if paused:
                        paused = False
                        # ball served from current side
                        ball.reset(serve_from_top=serve_from_top)
                elif event.key == pygame.K_r:
                    # reset scores and restart
                    player_score = 0
                    opponent_score = 0
                    paused = True
                    ball.reset(serve_from_top=serve_from_top)

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] or keys[pygame.K_a]:
            player.move_left()
        if keys[pygame.K_RIGHT] or keys[pygame.K_d]:
            player.move_right()

        # AI opponent movement
        if ai_enabled and not paused:
            # track ball's x with some lag
            diff = ball.x - opponent.x
            opponent.x += diff * ai_reaction * (1 + abs(ball.vx)/10)
            # clamp
            if opponent.x - opponent.w // 2 < 0:
                opponent.x = opponent.w // 2
            if opponent.x + opponent.w // 2 > SCREEN_W:
                opponent.x = SCREEN_W - opponent.w // 2

        # update ball if game not paused
        if not paused:
            ball.update()
            # check collisions with paddles
            paddle_ball_collision(ball, player, from_top=False)
            paddle_ball_collision(ball, opponent, from_top=True)

            # Check scoring: ball goes past top or bottom
            if ball.y - ball.r <= 0:
                # passed opponent (player scored)
                player_score += 1
                paused = True
                serve_from_top = True  # next serve from top side
                ball.reset(serve_from_top=serve_from_top)
            elif ball.y + ball.r >= SCREEN_H:
                opponent_score += 1
                paused = True
                serve_from_top = False
                ball.reset(serve_from_top=serve_from_top)

            # minor speed increase over time
            if random.random() < 0.002:
                ball.vx *= 1.01
                ball.vy *= 1.01

        # draw background
        if table_img:
            screen.blit(table_img, (0, 0))
        else:
            screen.fill((12, 90, 35))
            # center line and borders
            pygame.draw.rect(screen, (20, 110, 50), (30, 30, SCREEN_W - 60, SCREEN_H - 60), border_radius=16)
            # net line
            pygame.draw.line(screen, (240, 240, 240), (0, SCREEN_H//2), (SCREEN_W, SCREEN_H//2), 2)

        # draw paddles and ball
        opponent.draw(screen)
        player.draw(screen)
        ball.draw(screen)

        # HUD
        draw_text_center(screen, f"{opponent_score}  —  {player_score}", 48, 36)
        info1 = "Space: Serve  |  A/D or ←/→: Move  |  R: Reset"
        font = pygame.font.SysFont("dejavusans", 16)
        screen.blit(font.render(info1, True, (220, 220, 220)), (12, SCREEN_H - 24))

        if paused:
            draw_text_center(screen, "PAUSED — Press SPACE to serve", 28, SCREEN_H // 2)

        # check for match win
        if player_score >= MAX_SCORE or opponent_score >= MAX_SCORE:
            paused = True
            if player_score > opponent_score:
                draw_text_center(screen, "YOU WIN! Press R to play again.", 34, SCREEN_H // 2 + 40, color=(200, 255, 200))
            else:
                draw_text_center(screen, "YOU LOSE. Press R to try again.", 34, SCREEN_H // 2 + 40, color=(255, 180, 180))

        pygame.display.flip()

    pygame.quit()
    sys.exit()


if __name__ == "__main__":
    main()


SystemExit: 

C:\Users\gagan\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
import os
import sys
import random
import math
import pygame

# SETTINGS
SCREEN_W, SCREEN_H = 1000, 600
FPS = 60
PADDLE_WIDTH = 160
PADDLE_HEIGHT = 160  # since racket is tall
BALL_RADIUS = 20
BALL_SPEED_INIT = 7.0
DOWNLOADS = os.path.expanduser("~/Downloads")

# Expected image files in Downloads
FILES = {
    "table": "C:\Users\gagan\Downloads\table.png",      # Real table image
    "player": "C:\Users\gagan\Downloads\plyaer.png",    # Real player image
    "racket": "C:\Users\gagan\Downloads\racket.jpeg",    # Real racket image
    "ball": "C:\Users\gagan\Downloads\ball.png"         # Real ball image
}


def load_image(name, size=None):
    """Try to load an image from Downloads folder"""
    path = os.path.join(DOWNLOADS, FILES[name])
    if not os.path.exists(path):
        print(f"[WARNING] {FILES[name]} not found in Downloads.")
        return None
    img = pygame.image.load(path).convert_alpha()
    if size:
        img = pygame.transform.smoothscale(img, size)
    return img


class Paddle:
    def __init__(self, x, y, image):
        self.x = x
        self.y = y
        self.speed = 9
        self.image = image
        self.rect = self.image.get_rect(center=(self.x, self.y))

    def update(self):
        self.rect.centerx = self.x
        self.rect.centery = self.y

    def draw(self, surface):
        surface.blit(self.image, self.rect)

    def move_left(self):
        self.x -= self.speed
        if self.x < 80:
            self.x = 80

    def move_right(self):
        self.x += self.speed
        if self.x > SCREEN_W - 80:
            self.x = SCREEN_W - 80


class Ball:
    def __init__(self, x, y, image):
        self.x = x
        self.y = y
        self.vx = 5
        self.vy = -BALL_SPEED_INIT
        self.image = image
        self.rect = self.image.get_rect(center=(self.x, self.y))

    def update(self):
        self.x += self.vx
        self.y += self.vy
        if self.x < 0 or self.x > SCREEN_W:
            self.vx *= -1
        self.rect.center = (self.x, self.y)

    def draw(self, surface):
        surface.blit(self.image, self.rect)


def main():
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_W, SCREEN_H))
    pygame.display.set_caption("Real Table Tennis Game")
    clock = pygame.time.Clock()

    # Load images
    table_img = load_image("table", (SCREEN_W, SCREEN_H))
    player_img = load_image("player", (200, 200))
    racket_img = load_image("racket", (PADDLE_WIDTH, PADDLE_HEIGHT))
    ball_img = load_image("ball", (BALL_RADIUS * 2, BALL_RADIUS * 2))

    if not all([table_img, player_img, racket_img, ball_img]):
        print("⚠️ Some images are missing. Please put table.png, player.png, racket.png, ball.png in Downloads.")
        pygame.quit()
        sys.exit()

    # Create objects
    player = Paddle(SCREEN_W // 2, SCREEN_H - 80, racket_img)
    opponent = Paddle(SCREEN_W // 2, 80, racket_img)
    ball = Ball(SCREEN_W // 2, SCREEN_H // 2, ball_img)

    running = True
    while running:
        screen.blit(table_img, (0, 0))

        for e in pygame.event.get():
            if e.type == pygame.QUIT:
                running = False

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            player.move_left()
        if keys[pygame.K_RIGHT]:
            player.move_right()

        # Opponent AI
        if opponent.x < ball.x:
            opponent.x += 5
        elif opponent.x > ball.x:
            opponent.x -= 5

        ball.update()

        # Collision with paddles
        if ball.rect.colliderect(player.rect):
            ball.vy *= -1
            ball.y = player.rect.top - BALL_RADIUS
        if ball.rect.colliderect(opponent.rect):
            ball.vy *= -1
            ball.y = opponent.rect.bottom + BALL_RADIUS

        # Draw everything
        player.update()
        opponent.update()
        player.draw(screen)
        opponent.draw(screen)
        ball.draw(screen)

        pygame.display.flip()
        clock.tick(FPS)

    pygame.quit()
    sys.exit()


if __name__ == "__main__":
    main()


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (361892303.py, line 18)

In [7]:
import pygame
import sys
import os

# Initialize pygame
pygame.init()

# Screen setup
WIDTH, HEIGHT = 1000, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Table Tennis Game")

# FPS
clock = pygame.time.Clock()
FPS = 60

# ✅ File paths (change these if your files have different names)
FILES = {
    "table": r"C:\Users\gagan\Downloads\table.png",
    "player": r"C:\Users\gagan\Downloads\player.png",
    "racket": r"C:\Users\gagan\Downloads\racket.jpeg",
    "ball": r"C:\Users\gagan\Downloads\ball.png"
}

def load_image(name, size=None):
    """Try to load an image from given path, fallback if missing"""
    path = FILES[name]
    if not os.path.exists(path):
        print(f"[WARNING] {path} not found. Using placeholder.")
        return None
    img = pygame.image.load(path).convert_alpha()
    if size:
        img = pygame.transform.smoothscale(img, size)
    return img

# ✅ Load images
table_img = load_image("table", (WIDTH, HEIGHT))
player_img = load_image("player", (100, 150))
racket_img = load_image("racket", (40, 80))
ball_img = load_image("ball", (30, 30))

# Fallback colors if image not found
FALLBACK_COLOR = {
    "player": (0, 128, 255),
    "racket": (255, 0, 0),
    "ball": (255, 255, 0)
}

# Positions
player_x, player_y = 50, HEIGHT // 2 - 75
opponent_x, opponent_y = WIDTH - 150, HEIGHT // 2 - 75
ball_x, ball_y = WIDTH // 2, HEIGHT // 2
ball_speed_x, ball_speed_y = 5, 5

# Game Loop
running = True
while running:
    screen.fill((0, 100, 0))  # background if table missing
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_UP]:
        player_y -= 6
    if keys[pygame.K_DOWN]:
        player_y += 6

    # Move ball
    ball_x += ball_speed_x
    ball_y += ball_speed_y

    # Bounce from top/bottom
    if ball_y <= 0 or ball_y >= HEIGHT - 30:
        ball_speed_y *= -1

    # Reset if ball goes out
    if ball_x <= 0 or ball_x >= WIDTH - 30:
        ball_x, ball_y = WIDTH // 2, HEIGHT // 2
        ball_speed_x *= -1

    # Draw table
    if table_img:
        screen.blit(table_img, (0, 0))

    # Draw player
    if player_img:
        screen.blit(player_img, (player_x, player_y))
    else:
        pygame.draw.rect(screen, FALLBACK_COLOR["player"], (player_x, player_y, 100, 150))

    # Draw opponent
    if player_img:
        screen.blit(player_img, (opponent_x, opponent_y))
    else:
        pygame.draw.rect(screen, FALLBACK_COLOR["player"], (opponent_x, opponent_y, 100, 150))

    # Draw rackets
    if racket_img:
        screen.blit(racket_img, (player_x + 90, player_y + 40))
        screen.blit(racket_img, (opponent_x - 40, opponent_y + 40))
    else:
        pygame.draw.rect(screen, FALLBACK_COLOR["racket"], (player_x + 90, player_y + 40, 40, 80))
        pygame.draw.rect(screen, FALLBACK_COLOR["racket"], (opponent_x - 40, opponent_y + 40, 40, 80))

    # Draw ball
    if ball_img:
        screen.blit(ball_img, (ball_x, ball_y))
    else:
        pygame.draw.circle(screen, FALLBACK_COLOR["ball"], (ball_x, ball_y), 15)

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()
sys.exit()


[WARNING] C:\Users\gagan\Downloads\player.png not found. Using placeholder.


SystemExit: 

In [8]:
"""
Real-feel Table Tennis (Ping-Pong) in Python using pygame

Features:
- Uses real images if present (Downloads folder) else draws shapes.
- Paddle rotation and spin that affects ball curve.
- Simple AI opponent with prediction.
- Scoring, serve, restart.
- Tweak constants for more realism.

Controls:
  - A / Left  : move left
  - D / Right : move right
  - Q         : rotate paddle counter-clockwise
  - E         : rotate paddle clockwise
  - Space     : serve / start
  - R         : reset scores
  - Esc / Close window : quit

Requires: pygame
Install: pip install pygame
"""

import os
import sys
import math
import random
import pygame

# ---------- SETTINGS ----------
SCREEN_W, SCREEN_H = 1100, 650
FPS = 60

# paddle sizes
PADDLE_W, PADDLE_H = 160, 36   # logical paddle (racket image will be scaled to this)
PADDLE_Y_OFFSET = 60

# ball
BALL_RADIUS = 12
BALL_INIT_SPEED = 7.5
BALL_MAX_SPEED = 18.0

# gameplay
MAX_SCORE = 11
WIN_BY = 2

# image file paths (edit if you store elsewhere)
DOWNLOADS = os.path.expanduser("~/Downloads")
FILES = {
    "table": os.path.join(DOWNLOADS, "table.png"),
    "racket": os.path.join(DOWNLOADS, "racket.png"),
    "ball": os.path.join(DOWNLOADS, "ball.png"),
}

# physics tuning
PADDLE_MOVE_SPEED = 9.5
PADDLE_ROT_SPEED = 2.5  # degrees per frame when pressing rotate keys
SPIN_FACTOR = 0.18      # how much paddle angle + offset generates spin
HIT_VELOCITY_BOOST = 1.08
NET_HEIGHT = 6          # in pixels (for collision effect)
TABLE_MARGIN = 50       # visual margin
GRAVITY = 0.0           # we don't simulate vertical arc strongly; leave near 0 for classic arcade feel

# AI settings
AI_REACTION = 0.14     # how fast AI adjusts horizontally
AI_LEAD = 8            # how far ahead AI tries to intercept horizontally

# colors
BG_COLOR = (18, 90, 36)
WHITE = (245, 245, 245)
SCORE_COLOR = (240, 240, 200)

# ---------- PYGAME SETUP ----------
pygame.init()
screen = pygame.display.set_mode((SCREEN_W, SCREEN_H))
pygame.display.set_caption("Real Table Tennis - Python / Pygame")
clock = pygame.time.Clock()
font_large = pygame.font.SysFont("dejavusans", 48, bold=True)
font_small = pygame.font.SysFont("dejavusans", 18)

# ---------- IMAGE LOADING with fallback ----------
def load_image(path, size=None, convert_alpha=True):
    if path and os.path.exists(path):
        try:
            img = pygame.image.load(path)
            if convert_alpha:
                img = img.convert_alpha()
            else:
                img = img.convert()
            if size:
                img = pygame.transform.smoothscale(img, size)
            return img
        except Exception as e:
            print(f"[WARN] Failed to load {path}: {e}")
            return None
    return None

table_img = load_image(FILES["table"], (SCREEN_W, SCREEN_H))
racket_img_orig = load_image(FILES["racket"], (PADDLE_W, PADDLE_H))
ball_img = load_image(FILES["ball"], (BALL_RADIUS*2, BALL_RADIUS*2))

# if no racket image, we will draw a rounded rect as paddle
USE_IMAGES = True if (table_img or racket_img_orig or ball_img) else False

# ---------- GAME OBJECTS ----------
class Paddle:
    def __init__(self, x, y, is_player=True):
        self.x = x
        self.y = y
        self.w = PADDLE_W
        self.h = PADDLE_H
        self.angle = 0.0         # degrees; 0 = horizontal
        self.is_player = is_player
        self.image = racket_img_orig
        self.rect = pygame.Rect(0,0,self.w, self.h)
        self.update_rect()

    def update_rect(self):
        self.rect.center = (int(self.x), int(self.y))

    def draw(self, surf):
        if self.image:
            # rotate image about center
            surf_img = pygame.transform.rotate(self.image, self.angle)
            r = surf_img.get_rect(center=(int(self.x), int(self.y)))
            surf.blit(surf_img, r.topleft)
            self.rect = r
        else:
            # fallback: draw a rounded rectangle and a small handle
            pad = pygame.Surface((self.w, self.h), pygame.SRCALPHA)
            pygame.draw.rect(pad, (220,220,220), (0,0,self.w,self.h), border_radius=8)
            pad = pygame.transform.rotate(pad, self.angle)
            r = pad.get_rect(center=(int(self.x), int(self.y)))
            surf.blit(pad, r.topleft)
            self.rect = r

    def move(self, dx):
        self.x += dx
        halfw = self.w//2 + 4
        if self.x - halfw < TABLE_MARGIN:
            self.x = TABLE_MARGIN + halfw
        if self.x + halfw > SCREEN_W - TABLE_MARGIN:
            self.x = SCREEN_W - TABLE_MARGIN - halfw

class Ball:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.r = BALL_RADIUS
        angle = random.uniform(-0.30*math.pi, 0.30*math.pi)
        # start upwards or downwards depending on serve state; we'll set velocity on serve
        self.vx = BALL_INIT_SPEED * math.sin(angle)
        self.vy = -BALL_INIT_SPEED * math.cos(angle)
        self.spin = 0.0  # positive = curve to the right
        self.image = ball_img

    def reset_position(self, serve_from_top=False):
        self.x = SCREEN_W//2
        self.y = SCREEN_H//2
        angle = random.uniform(-0.25*math.pi, 0.25*math.pi)
        if serve_from_top:
            self.vx = BALL_INIT_SPEED * math.sin(angle)
            self.vy = abs(BALL_INIT_SPEED * math.cos(angle))
        else:
            self.vx = BALL_INIT_SPEED * math.sin(angle)
            self.vy = -abs(BALL_INIT_SPEED * math.cos(angle))
        self.spin = 0.0

    def update(self, dt):
        # Apply spin: spin causes a lateral acceleration on vx
        self.vx += self.spin * SPIN_FACTOR * dt * 60  # scaled to frames
        # Gravity component if used (kept small)
        self.vy += GRAVITY * dt * 60

        # clamp speed magnitude
        speed = math.hypot(self.vx, self.vy)
        if speed > BALL_MAX_SPEED:
            scale = BALL_MAX_SPEED / speed
            self.vx *= scale
            self.vy *= scale

        self.x += self.vx * dt * 60
        self.y += self.vy * dt * 60

        # side walls
        if self.x - self.r <= TABLE_MARGIN:
            self.x = TABLE_MARGIN + self.r
            self.vx *= -1
            self.spin *= 0.6
        if self.x + self.r >= SCREEN_W - TABLE_MARGIN:
            self.x = SCREEN_W - TABLE_MARGIN - self.r
            self.vx *= -1
            self.spin *= 0.6

    def draw(self, surf):
        if self.image:
            surf_img = pygame.transform.smoothscale(self.image, (self.r*2, self.r*2))
            rect = surf_img.get_rect(center=(int(self.x), int(self.y)))
            surf.blit(surf_img, rect.topleft)
        else:
            pygame.draw.circle(surf, (240,240,240), (int(self.x), int(self.y)), self.r)

    def rect(self):
        return pygame.Rect(int(self.x - self.r), int(self.y - self.r), self.r*2, self.r*2)

# ---------- UTILS ----------
def draw_table(surf):
    if table_img:
        surf.blit(table_img, (0,0))
    else:
        surf.fill(BG_COLOR)
        # table inner rectangle
        inner = pygame.Rect(TABLE_MARGIN, TABLE_MARGIN, SCREEN_W - 2*TABLE_MARGIN, SCREEN_H - 2*TABLE_MARGIN)
        pygame.draw.rect(surf, (25,120,55), inner, border_radius=8)
        # center line (net line)
        pygame.draw.line(surf, (220,220,220), (TABLE_MARGIN, SCREEN_H//2), (SCREEN_W - TABLE_MARGIN, SCREEN_H//2), 2)
        # net (visual)
        net_y = SCREEN_H//2 - NET_HEIGHT//2
        pygame.draw.rect(surf, (200,200,200), (TABLE_MARGIN, net_y, SCREEN_W - 2*TABLE_MARGIN, NET_HEIGHT), 0)

def draw_hud(surf, score_p, score_o, paused, serve_from_top):
    text = font_large.render(f"{score_o}   —   {score_p}", True, SCORE_COLOR)
    surf.blit(text, (SCREEN_W//2 - text.get_width()//2, 12))
    if paused:
        sub = "PAUSED — Press SPACE to Serve" if serve_from_top is None else "PAUSED — Press SPACE to Serve"
        txt = font_small.render(sub + " | Q/E rotate | A/D move | R reset", True, (240,240,240))
        surf.blit(txt, (SCREEN_W//2 - txt.get_width()//2, 80))

# ---------- INITIAL GAME STATE ----------
player = Paddle(SCREEN_W//2, SCREEN_H - PADDLE_Y_OFFSET, is_player=True)
opponent = Paddle(SCREEN_W//2, PADDLE_Y_OFFSET, is_player=False)
ball = Ball(SCREEN_W//2, SCREEN_H//2)

player_score = 0
opponent_score = 0
paused = True
serve_from_top = False  # who serves next; False => bottom serves
ball.reset_position(serve_from_top=serve_from_top)

# extra gameplay state
rally_active = False
last_hit_by = None  # "player" or "opponent"

# ---------- COLLISION HELPERS ----------
def circle_rect_collision(circle: Ball, paddle: Paddle):
    # approximate circle-rect collision
    r = circle.r
    rect = paddle.rect
    nearest_x = max(rect.left, min(circle.x, rect.right))
    nearest_y = max(rect.top, min(circle.y, rect.bottom))
    dx = circle.x - nearest_x
    dy = circle.y - nearest_y
    return (dx*dx + dy*dy) <= r*r

def handle_paddle_hit(circle: Ball, paddle: Paddle, from_top: bool):
    # Called when collision detected. Reflect vertical velocity and add spin from paddle angle + hit offset.
    # Compute offset relative to paddle center
    offset = (circle.x - paddle.x) / (paddle.w / 2)  # -1 .. 1
    # base vertical reflect
    if from_top and circle.vy > 0:
        circle.y = paddle.rect.bottom + circle.r + 1
        circle.vy *= -1
    elif not from_top and circle.vy < 0:
        circle.y = paddle.rect.top - circle.r - 1
        circle.vy *= -1
    else:
        circle.vy *= -1

    # change vx based on where it was hit and paddle rotation
    # paddle.angle in degrees; convert to radians for contribution
    ang = math.radians(paddle.angle)
    # offset influence + rotation influence => vx change and spin change
    circle.vx += offset * 2.2 + math.sin(ang) * 2.5
    # add spin
    circle.spin += (offset * 2.0 + math.sin(ang) * 4.0) * (1 if paddle.is_player else -1)
    # small speed boost
    circle.vx *= HIT_VELOCITY_BOOST
    circle.vy *= HIT_VELOCITY_BOOST
    # clamp spin to a reasonable range
    circle.spin = max(min(circle.spin, 10.0), -10.0)
    # ensure ball moves away
    if from_top:
        if circle.vy < 0:
            circle.vy = -circle.vy
    else:
        if circle.vy > 0:
            circle.vy = -circle.vy

# ---------- AI: Predict and move ----------
def ai_update(op, ball_obj, dt):
    # naive prediction: assume current vx/vy linear, try to position to intercept
    # we only predict horizontal landing where ball will cross ai's y position
    if ball_obj.vy == 0:
        target_x = SCREEN_W//2
    else:
        # predict time to reach opponent's y
        dy = op.y - ball_obj.y
        t = None
        if ball_obj.vy != 0:
            t = dy / (ball_obj.vy)  # time in frames (since velocities scaled per frame)
        if t is None or t <= 0:
            target_x = ball_obj.x
        else:
            # linear predict
            target_x = ball_obj.x + ball_obj.vx * (t)
            # account for wall reflections (mirror in ranges)
            left = TABLE_MARGIN + ball_obj.r
            right = SCREEN_W - TABLE_MARGIN - ball_obj.r
            # reflect into [left, right] interval
            span = right - left
            if span > 0:
                # fold
                while target_x < left or target_x > right:
                    if target_x < left:
                        target_x = left + (left - target_x)
                    if target_x > right:
                        target_x = right - (target_x - right)
    # move towards target with some lag
    # lead offset so AI doesn't sit exactly at ball.x
    if target_x > op.x + AI_LEAD:
        op.x += PADDLE_MOVE_SPEED * AI_REACTION * (1 + abs(ball_obj.vx)/10)
    elif target_x < op.x - AI_LEAD:
        op.x -= PADDLE_MOVE_SPEED * AI_REACTION * (1 + abs(ball_obj.vx)/10)
    # clamp
    halfw = op.w//2 + 4
    if op.x - halfw < TABLE_MARGIN:
        op.x = TABLE_MARGIN + halfw
    if op.x + halfw > SCREEN_W - TABLE_MARGIN:
        op.x = SCREEN_W - TABLE_MARGIN - halfw

# ---------- MAIN GAME LOOP ----------
running = True
while running:
    dt = clock.tick(FPS) / 1000.0  # seconds per frame
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            elif event.key == pygame.K_SPACE:
                if paused:
                    paused = False
                    rally_active = True
                    # serve: push ball towards opponent or player based on serve_from_top
                    # small deterministic serve:
                    angle = random.uniform(-0.2*math.pi, 0.2*math.pi)
                    if serve_from_top:
                        ball.vx = BALL_INIT_SPEED * math.sin(angle)
                        ball.vy = abs(BALL_INIT_SPEED * math.cos(angle))
                    else:
                        ball.vx = BALL_INIT_SPEED * math.sin(angle)
                        ball.vy = -abs(BALL_INIT_SPEED * math.cos(angle))
                    ball.spin = 0.0
                    last_hit_by = None
                else:
                    # mid rally: allow to pause
                    paused = True
            elif event.key == pygame.K_r:
                # reset score and state
                player_score = 0
                opponent_score = 0
                paused = True
                serve_from_top = False
                ball.reset_position(serve_from_top=serve_from_top)
                rally_active = False
                last_hit_by = None

    # Input handling
    keys = pygame.key.get_pressed()
    move_dx = 0
    if keys[pygame.K_a] or keys[pygame.K_LEFT]:
        move_dx -= PADDLE_MOVE_SPEED
    if keys[pygame.K_d] or keys[pygame.K_RIGHT]:
        move_dx += PADDLE_MOVE_SPEED
    player.move(move_dx * (1 if dt > 0 else 0))

    if keys[pygame.K_q]:
        player.angle = max(player.angle - PADDLE_ROT_SPEED, -60)
    elif keys[pygame.K_e]:
        player.angle = min(player.angle + PADDLE_ROT_SPEED, 60)
    else:
        # slowly recover angle to 0 to simulate natural racket movement
        player.angle *= 0.98

    # Opponent angle auto-adjust to face ball
    # simple heuristic: angle towards ball
    dx_to_ball = ball.x - opponent.x
    target_ang = max(min(dx_to_ball / 5.0, 40), -40)
    opponent.angle += (target_ang - opponent.angle) * 0.06

    # AI movement
    if not paused:
        ai_update(opponent, ball, dt)

    # Update ball if rally active
    if rally_active and not paused:
        ball.update(dt)

        # collisions with paddles
        if circle_rect_collision(ball, player):
            handle_paddle_hit(ball, player, from_top=False)
            last_hit_by = "player"

        if circle_rect_collision(ball, opponent):
            handle_paddle_hit(ball, opponent, from_top=True)
            last_hit_by = "opponent"

        # check net collision (very simple): if ball is near centerline and low enough, it can hit net
        net_y = SCREEN_H//2
        if abs(ball.y - net_y) < ball.r + NET_HEIGHT:
            # check if crossing and too low -> net hit
            # if ball is between table margins horizontally near center when crossing
            if TABLE_MARGIN < ball.x < SCREEN_W - TABLE_MARGIN:
                # if ball is lower than net top, it hits net and the side who hit loses the point
                # we interpret net as blocking if ball y is within net zone
                # Determine who last hit — they lose the rally when hitting net
                if last_hit_by == "player" and ball.vy < 0:
                    opponent_score += 1
                    paused = True
                    rally_active = False
                    serve_from_top = True
                    ball.reset_position(serve_from_top=serve_from_top)
                elif last_hit_by == "opponent" and ball.vy > 0:
                    player_score += 1
                    paused = True
                    rally_active = False
                    serve_from_top = False
                    ball.reset_position(serve_from_top=serve_from_top)

        # check if ball went past top or bottom (i.e., missed)
        if ball.y - ball.r <= 0:
            # ball passed opponent side -> player scores
            player_score += 1
            paused = True
            rally_active = False
            serve_from_top = True
            ball.reset_position(serve_from_top=serve_from_top)
            last_hit_by = None
        elif ball.y + ball.r >= SCREEN_H:
            # opponent scores
            opponent_score += 1
            paused = True
            rally_active = False
            serve_from_top = False
            ball.reset_position(serve_from_top=serve_from_top)
            last_hit_by = None

    # draw everything
    draw_table(screen)
    # draw net visually (a bit darker)
    net_rect = pygame.Rect(TABLE_MARGIN, SCREEN_H//2 - NET_HEIGHT//2, SCREEN_W - 2*TABLE_MARGIN, NET_HEIGHT)
    pygame.draw.rect(screen, (210,210,210), net_rect)

    opponent.update_rect()
    player.update_rect()
    opponent.draw(screen)
    player.draw(screen)

    ball.draw(screen)
    draw_hud(screen, player_score, opponent_score, paused, serve_from_top)

    # Check match win (with win-by-two)
    if (player_score >= MAX_SCORE or opponent_score >= MAX_SCORE) and abs(player_score - opponent_score) >= WIN_BY:
        paused = True
        rally_active = False
        winner_text = "YOU WIN!" if player_score > opponent_score else "YOU LOSE!"
        txt = font_large.render(winner_text + " Press R to play again.", True, (220, 240, 200))
        screen.blit(txt, (SCREEN_W//2 - txt.get_width()//2, SCREEN_H//2 - 20))

    pygame.display.flip()

pygame.quit()
sys.exit()


SystemExit: 

In [9]:
"""
Realistic-looking Table Tennis (Pygame)
- 3D-ish ball arc (z height) with shadow + scale
- Racket swing animation (visual)
- Ball motion blur (trail)
- Uses real images if present (Downloads folder)
"""

import os, sys, math, random, pygame
from collections import deque

# ---------------- CONFIG ----------------
SCREEN_W, SCREEN_H = 1200, 700
FPS = 60

DOWNLOADS = r"C:\Users\gagan\Downloads"  # <- change if needed
FILES = {
    "table": os.path.join(DOWNLOADS, "table.png"),
    "player": os.path.join(DOWNLOADS, "player.png"),
    "racket": os.path.join(DOWNLOADS, "racket.png"),
    "ball": os.path.join(DOWNLOADS, "ball.png"),
}

TABLE_MARGIN = 90
NET_Y = SCREEN_H // 2
NET_HEIGHT = 6

# paddle visual config (logical)
PADDLE_W, PADDLE_H = 140, 38
PADDLE_OFFSET = 80  # vertical offset from edge

# ball physics (3D-ish)
BALL_RADIUS_BASE = 10   # visible radius at z=0 (on table)
BALL_INIT_SPEED = 8.5
BALL_MAX_SPEED = 22.0
GRAVITY = 0.45          # influences z-arc

# gameplay
PADDLE_SPEED = 9.0
AI_SPEED = 7.0
MAX_SCORE = 11

# visuals
BG_COLOR = (16, 90, 40)
TRAIL_LENGTH = 10  # ball trail blur length

# --------------- PYGAME SETUP ---------------
pygame.init()
screen = pygame.display.set_mode((SCREEN_W, SCREEN_H))
pygame.display.set_caption("Realistic Table Tennis")
clock = pygame.time.Clock()
font_large = pygame.font.SysFont("Arial", 44, bold=True)
font_small = pygame.font.SysFont("Arial", 16)

# -------------- IMAGE LOADING ---------------
def load_img(path, size=None):
    if path and os.path.exists(path):
        try:
            img = pygame.image.load(path).convert_alpha()
            if size:
                img = pygame.transform.smoothscale(img, size)
            return img
        except Exception as e:
            print("Failed loading", path, e)
            return None
    return None

table_img = load_img(FILES["table"], (SCREEN_W, SCREEN_H))
player_img = load_img(FILES["player"], (160, 220))   # fallback scaled size
racket_img = load_img(FILES["racket"], (PADDLE_W, PADDLE_H))
ball_img = load_img(FILES["ball"], (BALL_RADIUS_BASE*2, BALL_RADIUS_BASE*2))

# fallback flags
USE_IMAGES = any([table_img, player_img, racket_img, ball_img])

# --------------- GAME OBJECTS ---------------
class Racket:
    def __init__(self, x, y, img=None, is_player=True):
        self.x = x
        self.y = y
        self.img = img
        self.angle = 0.0
        self.is_player = is_player
        self.swinging = False
        self.swing_timer = 0.0
        self.swing_duration = 0.18  # seconds
        self.swing_dir = 1  # 1 forward, -1 back
        self.offset_x = 0
        self.offset_y = 0

    def start_swing(self, power=1.0):
        if not self.swinging:
            self.swinging = True
            self.swing_timer = 0.0
            # randomize duration a bit for realism
            self.swing_duration = 0.14 + 0.06 * (1 - min(max(power,0),1))
            self.swing_dir = 1

    def update(self, dt):
        if self.swinging:
            self.swing_timer += dt
            t = self.swing_timer / max(self.swing_duration, 1e-6)
            # simple easing: back-swing -> forward -> recover
            if t < 0.6:
                # rotate forward quickly
                self.angle = (math.sin(t * math.pi * 1.3) * 55) * (1 if self.is_player else -1)
                self.offset_y = -6 * math.sin(t * math.pi)  # small vertical shift
                self.offset_x = 10 * math.sin(t * math.pi)
            else:
                # recover
                recovery = (t - 0.6) / 0.4
                self.angle *= max(0.0, 1 - recovery*1.1)
                self.offset_x *= max(0.0, 1 - recovery)
                self.offset_y *= max(0.0, 1 - recovery)
            if self.swing_timer >= self.swing_duration:
                # end swing
                self.swinging = False
                self.swing_timer = 0.0
                self.angle = 0.0
                self.offset_x = 0
                self.offset_y = 0

    def draw(self, surf):
        # draw racket image if present, rotated and offset
        if self.img:
            surf_img = pygame.transform.rotate(self.img, self.angle)
            r = surf_img.get_rect(center=(int(self.x + self.offset_x), int(self.y + self.offset_y)))
            surf.blit(surf_img, r.topleft)
        else:
            # fallback: draw rotated rectangle as racket
            surf_rect = pygame.Surface((PADDLE_W, PADDLE_H), pygame.SRCALPHA)
            pygame.draw.rect(surf_rect, (200,50,30), (0,0,PADDLE_W,PADDLE_H), border_radius=8)
            surf_img = pygame.transform.rotate(surf_rect, self.angle)
            r = surf_img.get_rect(center=(int(self.x + self.offset_x), int(self.y + self.offset_y)))
            surf.blit(surf_img, r.topleft)

class Player:
    def __init__(self, x, y, image=None, is_player=True):
        self.x = x
        self.y = y
        self.img = image
        self.racket = Racket(x + (40 if is_player else -40), y + 30, racket_img, is_player=is_player)
        self.is_player = is_player
        # small bobbing for realism
        self.bob = 0.0

    def update(self, dt):
        # keep racket following body
        self.racket.x = self.x + (40 if self.is_player else -40)
        self.racket.y = self.y + 30 + math.sin(self.bob) * 2
        self.bob += dt * 6.0
        self.racket.update(dt)

    def draw(self, surf):
        # draw body then racket in front/back appropriately
        if self.img:
            # scale slightly based on y for perspective
            surf_img = self.img
            r = surf_img.get_rect(center=(int(self.x), int(self.y)))
            surf.blit(surf_img, r.topleft)
        else:
            # fallback body
            pygame.draw.ellipse(surf, (80,160,220), (self.x - 30, self.y - 80, 60, 120))
        # draw racket in front
        self.racket.draw(surf)

class Ball3D:
    def __init__(self, x, y, vx=0.0, vy=0.0, vz=0.0):
        self.x = x
        self.y = y
        self.z = 0.0     # height above table surface (0 on table)
        self.vx = vx
        self.vy = vy
        self.vz = vz
        self.r = BALL_RADIUS_BASE
        self.trail = deque(maxlen=TRAIL_LENGTH)

    def apply_physics(self, dt):
        # integrate 2D position and z height
        self.vz -= GRAVITY * 60 * dt  # gravity acts downward in z
        self.x += self.vx * 60 * dt
        self.y += self.vy * 60 * dt
        self.z += self.vz * 60 * dt
        # bounce on table surface (z <= 0)
        if self.z <= 0:
            # simple restitution
            self.z = 0
            self.vz *= -0.62
            # slow in-plane speeds a bit to simulate friction
            self.vx *= 0.94
            self.vy *= 0.94
            # if small bounce, stop z
            if abs(self.vz) < 1.6:
                self.vz = 0.0

        # wall collisions in screen x domain (left/right)
        left = TABLE_MARGIN + self.screen_radius()
        right = SCREEN_W - TABLE_MARGIN - self.screen_radius()
        if self.x < left:
            self.x = left
            self.vx *= -1
        if self.x > right:
            self.x = right
            self.vx *= -1

        # append to trail (store projected positions)
        self.trail.append((self.x, self.y, self.z))

    def screen_radius(self):
        # scale radius small when high, larger when near table
        scale = 1.0 + (0.25 - (self.z / 200.0))
        return max(4, int(self.r * max(0.65, scale)))

    def draw(self, surf):
        # draw trail (simple faded circles)
        alpha = 180
        i = 0
        for tx, ty, tz in reversed(self.trail):
            s = max(2, int(self.r * (1.0 + tz / 150.0)) )
            surf_s = pygame.Surface((s*2, s*2), pygame.SRCALPHA)
            a = int(alpha * (1 - i / max(1, len(self.trail))))
            pygame.draw.circle(surf_s, (255,255,255,a), (s,s), s)
            surf.blit(surf_s, (int(tx - s), int(ty - s - tz*0.15)))
            i += 1

        # draw shadow on table (darker when near table)
        shadow_w = max(6, int(self.r * 2 * (1.0 - min(self.z, 200)/400)))
        shadow_h = max(3, shadow_w // 3)
        shadow_surf = pygame.Surface((shadow_w*2, shadow_h*2), pygame.SRCALPHA)
        pygame.draw.ellipse(shadow_surf, (0,0,0,100), (0,0,shadow_w*2,shadow_h*2))
        surface_x = int(self.x - shadow_w)
        surface_y = int(self.y - shadow_h/2 + 2)  # keep shadow on table plane
        surf.blit(shadow_surf, (surface_x, surface_y))

        # finally draw ball (projected by z)
        br = self.screen_radius()
        draw_y = int(self.y - br - self.z*0.15)  # lift up by z
        if ball_img:
            img_s = pygame.transform.smoothscale(ball_img, (br*2, br*2))
            surf.blit(img_s, (int(self.x - br), draw_y))
        else:
            pygame.draw.circle(surf, (255,255,255), (int(self.x), draw_y + br), br)

    def rect2d(self):
        # approximate 2D rect for collision with racket projected positions
        br = self.screen_radius()
        draw_y = int(self.y - br - self.z*0.15)
        return pygame.Rect(int(self.x - br), draw_y, br*2, br*2)

# -------------- Helpers --------------
def draw_table(bg):
    if table_img:
        bg.blit(table_img, (0,0))
    else:
        bg.fill(BG_COLOR)
        inner = pygame.Rect(TABLE_MARGIN, TABLE_MARGIN, SCREEN_W - 2*TABLE_MARGIN, SCREEN_H - 2*TABLE_MARGIN)
        pygame.draw.rect(bg, (18,110,60), inner, border_radius=10)
        # center line and net stripe
        pygame.draw.line(bg, (230,230,230), (TABLE_MARGIN, NET_Y), (SCREEN_W-TABLE_MARGIN, NET_Y), 2)

def draw_net(surf):
    net_rect = pygame.Rect(TABLE_MARGIN, NET_Y - NET_HEIGHT//2, SCREEN_W - 2*TABLE_MARGIN, NET_HEIGHT)
    pygame.draw.rect(surf, (230,230,230), net_rect)

# --------------- INITIAL STATE ---------------
player = Player(SCREEN_W//2, SCREEN_H - PADDLE_OFFSET, player_img, is_player=True)
opponent = Player(SCREEN_W//2, PADDLE_OFFSET, player_img, is_player=False)
ball = Ball3D(SCREEN_W//2, SCREEN_H//2, vx=0.0, vy=0.0, vz=0.0)

player_score = 0
opponent_score = 0
paused = True
serve_from_top = False
rally = False

# -------------- Gameplay utilities --------------
def serve_ball(from_top=False):
    # place ball near serving side, give upward initial vz
    if from_top:
        ball.x = opponent.x
        ball.y = opponent.y + 80
        ball.z = 0.0
        angle = random.uniform(-0.28, 0.28)
        speed = BALL_INIT_SPEED
        ball.vx = speed * math.sin(angle)
        ball.vy = speed * math.cos(angle)
        ball.vz = 10.5  # initial upward
    else:
        ball.x = player.x
        ball.y = player.y - 80
        ball.z = 0.0
        angle = random.uniform(-0.28, 0.28)
        speed = BALL_INIT_SPEED
        ball.vx = speed * math.sin(angle)
        ball.vy = -speed * math.cos(angle)
        ball.vz = 10.5
    ball.trail.clear()

def distance(a,b):
    return math.hypot(a.x-b.x, a.y-b.y)

def racket_hits_ball(racket:Racket, ball_obj:Ball3D):
    # check proximity of racket tip to ball projected
    # compute racket tip position (approx)
    rad = math.radians(racket.angle)
    tip_x = racket.x + ( (20 if racket.is_player else -20) + racket.offset_x ) + math.cos(rad) * (PADDLE_W//2)
    tip_y = racket.y + racket.offset_y - math.sin(rad) * (PADDLE_W//2)
    # approximate 2D distance
    bx = ball_obj.x
    by = ball_obj.y - ball_obj.z*0.15
    d = math.hypot(bx - tip_x, by - tip_y)
    # also consider z height to be close
    return d < 40 and ball_obj.z < 80

def hit_response(racket:Racket, ball_obj:Ball3D):
    # when racket hits ball: set vx, vy, vz based on racket angle and swing
    power = 1.0 + min(1.2, 0.6 + 0.4 * (racket.swing_timer / max(racket.swing_duration,1e-6)) ) if racket.swinging else 1.0
    # direction: shoot away from racket (upwards if player hits upward)
    dir_x = math.cos(math.radians(racket.angle)) * (1 if racket.is_player else -1)
    dir_y = -math.sin(math.radians(racket.angle))
    # mix with relative vec from racket to ball
    relx = (ball_obj.x - racket.x) * 0.09
    rely = (ball_obj.y - racket.y) * 0.09
    ball_obj.vx = dir_x * (BALL_INIT_SPEED * (1.0 + 0.6*power)) + relx
    ball_obj.vy = dir_y * (BALL_INIT_SPEED * (1.0 + 0.6*power)) + rely
    ball_obj.vz = 9.0 + 4.0 * power
    # small spin from angle and offset
    spin = math.sin(math.radians(racket.angle)) * (1.6 * (1 if racket.is_player else -1))
    ball_obj.vx += spin
    # record trail
    ball_obj.trail.clear()

# --------------- AI ---------------
def ai_move(opponent:Player, ball_obj:Ball3D, dt):
    # simple intercept: aim to align x to predicted x where ball crosses opponent's y
    if ball_obj.vy == 0:
        return
    target_x = ball_obj.x
    # predict time to opponent y in frames
    dy = opponent.y - ball_obj.y
    t = None
    if ball_obj.vy != 0:
        t = dy / (ball_obj.vy)
    if t is None or t <= 0:
        target_x = ball_obj.x
    else:
        target_x = ball_obj.x + ball_obj.vx * t
    # move towards target with limited speed
    if target_x > opponent.x + 8:
        opponent.x += AI_SPEED * dt * 60
    elif target_x < opponent.x - 8:
        opponent.x -= AI_SPEED * dt * 60
    # clamp inside table margins
    half = 60
    lo = TABLE_MARGIN + half
    hi = SCREEN_W - TABLE_MARGIN - half
    opponent.x = max(lo, min(hi, opponent.x))
    # if near ball and z low: attempt a swing
    # simple heuristic: if ball incoming and near
    if abs(ball_obj.x - opponent.x) < 80 and ball_obj.y < NET_Y + 90 and ball_obj.y > 40 and ball_obj.z < 70:
        # start swing sometimes
        if not opponent.racket.swinging and random.random() < 0.02:
            opponent.racket.start_swing()

# --------------- MAIN LOOP ---------------
serve_ball(serve_from_top)
running = True
while running:
    dt = clock.tick(FPS) / 1000.0
    for ev in pygame.event.get():
        if ev.type == pygame.QUIT:
            running = False
        elif ev.type == pygame.KEYDOWN:
            if ev.key == pygame.K_ESCAPE:
                running = False
            elif ev.key == pygame.K_SPACE:
                # if paused -> serve; else swing/hit
                if paused:
                    paused = False
                    rally = True
                    serve_ball(serve_from_top)
                else:
                    # trigger player swing (hit)
                    player.racket.start_swing()
            elif ev.key == pygame.K_r:
                player_score = 0
                opponent_score = 0
                paused = True
                rally = False
                serve_ball(False)
            elif ev.key == pygame.K_s:
                # toggle who serves
                serve_from_top = not serve_from_top

    keys = pygame.key.get_pressed()
    if keys[pygame.K_a] or keys[pygame.K_LEFT]:
        player.x -= PADDLE_SPEED
    if keys[pygame.K_d] or keys[pygame.K_RIGHT]:
        player.x += PADDLE_SPEED
    # rotate racket manually
    if keys[pygame.K_q]:
        player.racket.angle = max(-65, player.racket.angle - 2.5)
    elif keys[pygame.K_e]:
        player.racket.angle = min(65, player.racket.angle + 2.5)
    else:
        # relax angle slowly
        player.racket.angle *= 0.96

    # clamp player inside table
    half = 80
    lo = TABLE_MARGIN + half
    hi = SCREEN_W - TABLE_MARGIN - half
    player.x = max(lo, min(hi, player.x))

    # update AI
    if not paused:
        ai_move(opponent, ball, dt)

    # update players
    player.update(dt)
    opponent.update(dt)

    # update rackets (animations handled inside)
    player.racket.update(dt)
    opponent.racket.update(dt)

    # ball physics
    if rally and not paused:
        ball.apply_physics(dt)
        # check racket hits: player first then opponent
        if racket_hits_ball(player.racket, ball):
            hit_response(player.racket, ball)
            player.racket.start_swing()
        if racket_hits_ball(opponent.racket, ball):
            hit_response(opponent.racket, ball)
            opponent.racket.start_swing()

        # check out of bounds beyond top/bottom -> point
        if ball.y - ball.z*0.15 <= 0:
            # ball out on opponent's end => player scores
            player_score += 1
            paused = True
            rally = False
            serve_from_top = True
            serve_ball(serve_from_top)
        elif ball.y + ball.z*0.15 >= SCREEN_H:
            opponent_score += 1
            paused = True
            rally = False
            serve_from_top = False
            serve_ball(serve_from_top)

    # DRAW
    draw_table(screen)
    draw_net(screen)

    # draw opponent behind ball sometimes (depending on y)
    # draw order: farther y (smaller y) behind
    # we want proper layer: draw objects sorted by y
    entities = []
    entities.append(("opponent_body", opponent.y, opponent))
    entities.append(("player_body", player.y, player))
    entities.append(("ball", ball.y - ball.z*0.15, ball))
    # sort by y
    entities_sorted = sorted(entities, key=lambda x: x[1])
    for name, _, obj in entities_sorted:
        if name == "opponent_body":
            obj.draw(screen)
        elif name == "player_body":
            obj.draw(screen)
        elif name == "ball":
            obj.draw(screen)

    # HUD and overlay
    score_surf = font_large.render(f"{opponent_score}   —   {player_score}", True, (245,245,220))
    screen.blit(score_surf, (SCREEN_W//2 - score_surf.get_width()//2, 18))
    hint = "A/D or ←/→ move  •  Q/E rotate  •  SPACE serve/hit  •  R reset"
    screen.blit(font_small.render(hint, True, (230,230,230)), (20, SCREEN_H - 30))

    if paused:
        info = "PAUSED — Press SPACE to serve" if not rally else "PAUSED"
        screen.blit(font_small.render(info, True, (250,250,250)), (SCREEN_W//2-120, SCREEN_H//2-10))

    # Check win
    if (player_score >= MAX_SCORE or opponent_score >= MAX_SCORE) and abs(player_score - opponent_score) >= 2:
        winner = "YOU WIN!" if player_score > opponent_score else "YOU LOSE"
        big = font_large.render(winner + "  (R to restart)", True, (200,255,200))
        screen.blit(big, (SCREEN_W//2 - big.get_width()//2, SCREEN_H//2 + 30))

    pygame.display.flip()

pygame.quit()
sys.exit()


SystemExit: 

In [10]:
import pygame
import sys
import os
import random

# Initialize Pygame
pygame.init()

# Screen setup
WIDTH, HEIGHT = 1000, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Real Table Tennis Game")

# Colors
WHITE = (255, 255, 255)

# Load images (make sure these exist in the folder)
def load_image(path, fallback_color, size):
    if os.path.exists(path):
        img = pygame.image.load(path)
        img = pygame.transform.scale(img, size)
    else:
        img = pygame.Surface(size)
        img.fill(fallback_color)
    return img

# Table
table = load_image("table.png", (50, 150, 50), (WIDTH, HEIGHT))

# Players
player_img = load_image("player.png", (0, 0, 255), (80, 160))
opponent_img = load_image("opponent.png", (255, 0, 0), (80, 160))

# Rackets
racket_img = load_image("racket.png", (200, 0, 0), (40, 80))

# Ball
ball_img = load_image("ball.png", (255, 255, 255), (30, 30))

# Player positions
player_x, player_y = 100, HEIGHT//2 - 80
opponent_x, opponent_y = WIDTH - 180, HEIGHT//2 - 80

# Ball position and speed (reduced speed here 👇)
ball_x, ball_y = WIDTH//2, HEIGHT//2
ball_dx, ball_dy = 3 * random.choice([-1, 1]), 3 * random.choice([-1, 1])  # Slower speed

# Clock
clock = pygame.time.Clock()

# Game loop
while True:
    screen.fill(WHITE)

    # Events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    # Player movement
    keys = pygame.key.get_pressed()
    if keys[pygame.K_UP] and player_y > 0:
        player_y -= 5
    if keys[pygame.K_DOWN] and player_y < HEIGHT - 160:
        player_y += 5

    # Opponent movement (simple AI)
    if opponent_y + 80 < ball_y:
        opponent_y += 4
    if opponent_y + 80 > ball_y:
        opponent_y -= 4

    # Ball movement
    ball_x += ball_dx
    ball_y += ball_dy

    # Bounce on top/bottom
    if ball_y <= 0 or ball_y >= HEIGHT - 30:
        ball_dy *= -1

    # Collision with player
    if (player_x < ball_x < player_x + 80 and player_y < ball_y < player_y + 160):
        ball_dx *= -1

    # Collision with opponent
    if (opponent_x < ball_x < opponent_x + 80 and opponent_y < ball_y < opponent_y + 160):
        ball_dx *= -1

    # Reset if ball goes out
    if ball_x <= 0 or ball_x >= WIDTH:
        ball_x, ball_y = WIDTH//2, HEIGHT//2
        ball_dx, ball_dy = 3 * random.choice([-1, 1]), 3 * random.choice([-1, 1])

    # Draw everything
    screen.blit(table, (0, 0))
    screen.blit(player_img, (player_x, player_y))
    screen.blit(racket_img, (player_x + 70, player_y + 40))
    screen.blit(opponent_img, (opponent_x, opponent_y))
    screen.blit(racket_img, (opponent_x - 30, opponent_y + 40))
    screen.blit(ball_img, (ball_x, ball_y))

    pygame.display.flip()
    clock.tick(60)


SystemExit: 

In [13]:
"""
Real-feel Table Tennis (Ping-Pong) in Python using pygame

Features:
- 3D-ish ball arc (z height) with shadow + scale
- Simplified spin (Magnus-like lateral acceleration)
- Paddle swing animation; hit depends on paddle angle + hit offset
- Simple AI opponent that moves & swings
- Uses real images automatically if present in Downloads, else falls back to drawn shapes
- Slower default ball speed tuned to feel realistic

Controls:
- A / ← : move left
- D / → : move right
- SPACE : serve / attempt a hit (swing)
- Q / E : rotate paddle (adds spin/angle)
- R : reset scores
- Esc or close window : quit

Author: ChatGPT (adapted)
"""

import os
import math
import random
import pygame
from collections import deque

# ---------------- CONFIG ----------------
SCREEN_W, SCREEN_H = 1180, 700
FPS = 60

# Where to look for images by default (change to your folder if needed)
DOWNLOADS = os.path.expanduser("~/Downloads")
FILES = {
    "table": os.path.join(DOWNLOADS, "table.png"),
    "player": os.path.join(DOWNLOADS, "player.png"),
    "racket": os.path.join(DOWNLOADS, "racket.png"),
    "ball": os.path.join(DOWNLOADS, "ball.png"),
}

# Table and visual margins
TABLE_MARGIN = 90
NET_Y = SCREEN_H // 2
NET_HEIGHT = 6

# Paddle (logical)
PADDLE_W, PADDLE_H = 140, 36
PADDLE_OFFSET = 80

# Ball physics (slower default speed)
BALL_RADIUS_BASE = 10
BALL_INIT_SPEED = 6        # slower initial speed for realistic feel
BALL_MAX_SPEED = 12
GRAVITY = 0.60               # affects z arc
MAGNUS_FACTOR = 0.025        # spin -> lateral acceleration (small)

# Movement / gameplay tuning
PADDLE_SPEED = 5
AI_SPEED = 4
TRAIL_LENGTH = 10
MAX_SCORE = 15
WIN_BY = 2

# Colors
BG_COLOR = (14, 80, 35)
TABLE_COLOR = (22, 120, 60)
NET_COLOR = (230, 230, 230)
SCORE_COLOR = (245, 245, 220)

# ---------------- PYGAME SETUP ----------------
pygame.init()
screen = pygame.display.set_mode((SCREEN_W, SCREEN_H))
pygame.display.set_caption("Real-feel Table Tennis")
clock = pygame.time.Clock()
font_large = pygame.font.SysFont("dejavusans", 44, bold=True)
font_small = pygame.font.SysFont("dejavusans", 16)

# ---------------- IMAGE LOADING ----------------
def load_img(path, size=None):
    if path and os.path.exists(path):
        try:
            img = pygame.image.load(path).convert_alpha()
            if size:
                img = pygame.transform.smoothscale(img, size)
            return img
        except Exception as e:
            print(f"[WARN] Failed to load {path}: {e}")
    return None

table_img = load_img(FILES["table"], (SCREEN_W, SCREEN_H))
player_img = load_img(FILES["player"], (160, 220))
racket_img = load_img(FILES["racket"], (PADDLE_W, PADDLE_H))
ball_img = load_img(FILES["ball"], (BALL_RADIUS_BASE*2, BALL_RADIUS_BASE*2))

# ---------------- GAME OBJECTS ----------------
class Racket:
    def __init__(self, x, y, img=None, is_player=True):
        self.x = x
        self.y = y
        self.img = img
        self.angle = 0.0
        self.is_player = is_player
        self.swinging = False
        self.swing_timer = 0.0
        self.swing_duration = 0.16
        self.offset_x = 0
        self.offset_y = 0

    def start_swing(self):
        if not self.swinging:
            self.swinging = True
            self.swing_timer = 0.0
            self.swing_duration = 0.12 + random.random()*0.08

    def update(self, dt):
        if self.swinging:
            self.swing_timer += dt
            t = self.swing_timer / max(self.swing_duration, 1e-6)
            if t < 0.6:
                # forward swing
                self.angle = (math.sin(t * math.pi) * 55) * (1 if self.is_player else -1)
                self.offset_x = math.sin(t * math.pi) * 12 * (1 if self.is_player else -1)
                self.offset_y = -6 * math.sin(t * math.pi)
            else:
                # recover
                rec = (t - 0.6) / 0.4
                self.angle *= max(0.0, 1 - rec*1.2)
                self.offset_x *= max(0.0, 1 - rec)
                self.offset_y *= max(0.0, 1 - rec)
            if self.swing_timer >= self.swing_duration:
                self.swinging = False
                self.swing_timer = 0.0
                self.angle = 0.0
                self.offset_x = 0
                self.offset_y = 0

    def draw(self, surf):
        if self.img:
            surf_img = pygame.transform.rotate(self.img, self.angle)
            r = surf_img.get_rect(center=(int(self.x + self.offset_x), int(self.y + self.offset_y)))
            surf.blit(surf_img, r.topleft)
            self.rect = r
        else:
            surf_rect = pygame.Surface((PADDLE_W, PADDLE_H), pygame.SRCALPHA)
            pygame.draw.rect(surf_rect, (220,180,60) if self.is_player else (200,60,40), (0,0,PADDLE_W,PADDLE_H), border_radius=8)
            surf_img = pygame.transform.rotate(surf_rect, self.angle)
            r = surf_img.get_rect(center=(int(self.x + self.offset_x), int(self.y + self.offset_y)))
            surf.blit(surf_img, r.topleft)
            self.rect = r

class Player:
    def __init__(self, x, y, body_img=None, is_player=True):
        self.x = x
        self.y = y
        self.img = body_img
        self.racket = Racket(x + (40 if is_player else -40), y + 30, racket_img, is_player=is_player)
        self.is_player = is_player
        self.bob = 0.0

    def update(self, dt):
        self.racket.x = self.x + (40 if self.is_player else -40)
        self.racket.y = self.y + 30 + math.sin(self.bob) * 2
        self.bob += dt * 5.5
        self.racket.update(dt)

    def draw_body(self, surf):
        # draw body (behind or ahead depends on layering)
        if self.img:
            r = self.img.get_rect(center=(int(self.x), int(self.y)))
            surf.blit(self.img, r.topleft)
        else:
            pygame.draw.ellipse(surf, (80,160,220) if self.is_player else (210,120,120), (self.x-28, self.y-78, 56, 120))

class Ball3D:
    def __init__(self, x, y, vx=0.0, vy=0.0, vz=0.0):
        self.x = x
        self.y = y
        self.z = 0.0
        self.vx = vx
        self.vy = vy
        self.vz = vz
        self.r = BALL_RADIUS_BASE
        self.trail = deque(maxlen=TRAIL_LENGTH)

    def apply_physics(self, dt):
        # spin-induced lateral acceleration (approximate)
        # spin stored in vx spin attribute? We'll handle spin with vx change from hits
        self.vz -= GRAVITY * 60 * dt
        self.x += self.vx * 60 * dt
        self.y += self.vy * 60 * dt
        self.z += self.vz * 60 * dt

        # apply a small Magnus-style lateral force proportional to z-spin stored in self.spin if present
        # (we store spin as additional attribute if set externally)
        if hasattr(self, "spin") and self.spin != 0.0:
            self.vx += self.spin * MAGNUS_FACTOR * 60 * dt

        # bounce off the table (z <= 0)
        if self.z <= 0:
            self.z = 0
            self.vz *= -0.57
            self.vx *= 0.96
            self.vy *= 0.96
            # small damping of spin on bounce
            if hasattr(self, "spin"):
                self.spin *= 0.72
            if abs(self.vz) < 1.6:
                self.vz = 0.0

        # wall collisions left/right
        left = TABLE_MARGIN + self.screen_radius()
        right = SCREEN_W - TABLE_MARGIN - self.screen_radius()
        if self.x < left:
            self.x = left
            self.vx *= -1
            if hasattr(self, "spin"): self.spin *= 0.6
        if self.x > right:
            self.x = right
            self.vx *= -1
            if hasattr(self, "spin"): self.spin *= 0.6

        # record trail
        self.trail.append((self.x, self.y, self.z))

    def screen_radius(self):
        scale = 1.0 + (0.18 - (self.z / 240.0))
        return max(4, int(self.r * max(0.6, scale)))

    def draw(self, surf):
        # shadow (on table)
        shadow_w = max(6, int(self.r*2 * (1.0 - min(self.z, 200)/400)))
        shadow_h = max(3, shadow_w//3)
        shadow = pygame.Surface((shadow_w*2, shadow_h*2), pygame.SRCALPHA)
        pygame.draw.ellipse(shadow, (0,0,0,110), (0,0,shadow_w*2, shadow_h*2))
        surf.blit(shadow, (int(self.x-shadow_w), int(self.y-shadow_h/2 + 2)))

        # trail
        alpha = 180
        i = 0
        for tx, ty, tz in reversed(self.trail):
            s = max(2, int(self.r * (1.0 + tz / 160.0)))
            surf_s = pygame.Surface((s*2, s*2), pygame.SRCALPHA)
            a = int(alpha * (1 - i / max(1, len(self.trail))))
            pygame.draw.circle(surf_s, (255,255,255,a), (s, s), s)
            surf.blit(surf_s, (int(tx - s), int(ty - s - tz*0.15)))
            i += 1

        # ball
        br = self.screen_radius()
        draw_y = int(self.y - br - self.z*0.15)
        if ball_img:
            img_s = pygame.transform.smoothscale(ball_img, (br*2, br*2))
            surf.blit(img_s, (int(self.x - br), draw_y))
        else:
            pygame.draw.circle(surf, (255,255,255), (int(self.x), draw_y + br), br)

    def rect2d(self):
        br = self.screen_radius()
        draw_y = int(self.y - br - self.z*0.15)
        return pygame.Rect(int(self.x - br), draw_y, br*2, br*2)

# ---------------- HELPERS ----------------
def draw_table(surf):
    if table_img:
        surf.blit(table_img, (0,0))
    else:
        surf.fill(BG_COLOR)
        inner = pygame.Rect(TABLE_MARGIN, TABLE_MARGIN, SCREEN_W - 2*TABLE_MARGIN, SCREEN_H - 2*TABLE_MARGIN)
        pygame.draw.rect(surf, TABLE_COLOR, inner, border_radius=10)
        # center line & net visual
        pygame.draw.line(surf, (230,230,230), (TABLE_MARGIN, NET_Y), (SCREEN_W - TABLE_MARGIN, NET_Y), 2)

def draw_net(surf):
    net_r = pygame.Rect(TABLE_MARGIN, NET_Y - NET_HEIGHT//2, SCREEN_W - 2*TABLE_MARGIN, NET_HEIGHT)
    pygame.draw.rect(surf, NET_COLOR, net_r)

def serve_ball(ball_obj, from_top=False):
    if from_top:
        ball_obj.x = opponent.x
        ball_obj.y = opponent.y + 80
        ball_obj.z = 0.0
        ang = random.uniform(-0.28, 0.28)
        speed = BALL_INIT_SPEED
        ball_obj.vx = speed * math.sin(ang)
        ball_obj.vy = speed * math.cos(ang)
        ball_obj.vz = 9.0
        ball_obj.spin = 0.0
    else:
        ball_obj.x = player.x
        ball_obj.y = player.y - 80
        ball_obj.z = 0.0
        ang = random.uniform(-0.28, 0.28)
        speed = BALL_INIT_SPEED
        ball_obj.vx = speed * math.sin(ang)
        ball_obj.vy = -speed * math.cos(ang)
        ball_obj.vz = 9.0
        ball_obj.spin = 0.0
    ball_obj.trail.clear()

def racket_tip_pos(racket: Racket):
    rad = math.radians(racket.angle)
    tip_x = racket.x + ( (20 if racket.is_player else -20) + racket.offset_x ) + math.cos(rad) * (PADDLE_W//2)
    tip_y = racket.y + racket.offset_y - math.sin(rad) * (PADDLE_W//2)
    return tip_x, tip_y

def racket_hits_ball(racket: Racket, ball_obj: Ball3D):
    tip_x, tip_y = racket_tip_pos(racket)
    bx = ball_obj.x
    by = ball_obj.y - ball_obj.z*0.15
    d = math.hypot(bx - tip_x, by - tip_y)
    return d < 44 and ball_obj.z < 120

def hit_response(racket: Racket, ball_obj: Ball3D):
    # compute base direction using racket angle & whether player or opponent
    power = 1.0 + min(1.0, 0.5 * (racket.swing_timer / max(racket.swing_duration, 1e-6))) if racket.swinging else 1.0
    dir_x = math.cos(math.radians(racket.angle)) * (1 if racket.is_player else -1)
    dir_y = -math.sin(math.radians(racket.angle))
    relx = (ball_obj.x - racket.x) * 0.08
    rely = (ball_obj.y - racket.y) * 0.08
    ball_obj.vx = dir_x * (BALL_INIT_SPEED * (1.0 + 0.6*power)) + relx
    ball_obj.vy = dir_y * (BALL_INIT_SPEED * (1.0 + 0.6*power)) + rely
    ball_obj.vz = 8.0 + 3.0 * power
    # spin from angle and hit offset
    spin = math.sin(math.radians(racket.angle)) * (1.6 * (1 if racket.is_player else -1))
    # offset influence: hit more to left or right -> spin sign
    offset = (ball_obj.x - racket.x) / (PADDLE_W/2)
    spin += offset * 1.8 * (1 if racket.is_player else -1)
    ball_obj.spin = getattr(ball_obj, "spin", 0.0) + spin
    # small speed boost
    ball_obj.vx *= 1.02
    ball_obj.vy *= 1.02
    # clamp speeds
    mag = math.hypot(ball_obj.vx, ball_obj.vy)
    if mag > BALL_MAX_SPEED:
        scale = BALL_MAX_SPEED / mag
        ball_obj.vx *= scale
        ball_obj.vy *= scale

# ---------------- AI ----------------
def ai_update(opponent: Player, ball_obj: Ball3D, dt):
    # Predict x position where ball crosses opponent.y (linear approx without spin)
    if ball_obj.vy == 0:
        target_x = ball_obj.x
    else:
        dy = opponent.y - ball_obj.y
        t = None
        if ball_obj.vy != 0:
            t = dy / (ball_obj.vy)
        if t is None or t <= 0:
            target_x = ball_obj.x
        else:
            target_x = ball_obj.x + ball_obj.vx * t
            # reflect into table bounds
            left = TABLE_MARGIN + ball_obj.screen_radius()
            right = SCREEN_W - TABLE_MARGIN - ball_obj.screen_radius()
            span = right - left
            if span > 0:
                # fold into range
                while target_x < left or target_x > right:
                    if target_x < left:
                        target_x = left + (left - target_x)
                    if target_x > right:
                        target_x = right - (target_x - right)
    # move toward target
    if target_x > opponent.x + 8:
        opponent.x += AI_SPEED * dt * 60
    elif target_x < opponent.x - 8:
        opponent.x -= AI_SPEED * dt * 60
    # clamp
    half = 70
    opponent.x = max(TABLE_MARGIN + half, min(SCREEN_W - TABLE_MARGIN - half, opponent.x))
    # decide to swing if ball nearby and low
    if abs(ball_obj.x - opponent.x) < 80 and ball_obj.y < NET_Y + 90 and ball_obj.z < 80:
        if not opponent.racket.swinging and random.random() < 0.02:
            opponent.racket.start_swing()

# ---------------- INITIAL STATE ----------------
player = Player(SCREEN_W//2, SCREEN_H - PADDLE_OFFSET, player_img, is_player=True)
opponent = Player(SCREEN_W//2, PADDLE_OFFSET, player_img, is_player=False)
ball = Ball3D(SCREEN_W//2, SCREEN_H//2)
ball.spin = 0.0

player_score = 0
opponent_score = 0
paused = True
serve_from_top = False
rally = False
last_hit = None

serve_ball(ball, serve_from_top)

# ---------------- MAIN LOOP ----------------
running = True
while running:
    dt = clock.tick(FPS) / 1000.0
    for ev in pygame.event.get():
        if ev.type == pygame.QUIT:
            running = False
        elif ev.type == pygame.KEYDOWN:
            if ev.key == pygame.K_ESCAPE:
                running = False
            elif ev.key == pygame.K_SPACE:
                if paused:
                    paused = False
                    rally = True
                    serve_ball(ball, serve_from_top)
                else:
                    player.racket.start_swing()
            elif ev.key == pygame.K_r:
                player_score = 0
                opponent_score = 0
                paused = True
                rally = False
                serve_from_top = False
                serve_ball(ball, serve_from_top)
            elif ev.key == pygame.K_s:
                serve_from_top = not serve_from_top

    # Input: move player left/right
    keys = pygame.key.get_pressed()
    if keys[pygame.K_a] or keys[pygame.K_LEFT]:
        player.x -= PADDLE_SPEED
    if keys[pygame.K_d] or keys[pygame.K_RIGHT]:
        player.x += PADDLE_SPEED
    # rotate racket
    if keys[pygame.K_q]:
        player.racket.angle = max(-70, player.racket.angle - 2.5)
    elif keys[pygame.K_e]:
        player.racket.angle = min(70, player.racket.angle + 2.5)
    else:
        player.racket.angle *= 0.97

    # clamp player in table area
    half = 80
    player.x = max(TABLE_MARGIN + half, min(SCREEN_W - TABLE_MARGIN - half, player.x))

    # AI
    if not paused:
        ai_update(opponent, ball, dt)

    # update players & rackets
    player.update(dt)
    opponent.update(dt)

    # ball physics & collision if rally active
    if rally and not paused:
        ball.apply_physics(dt)
        # check racket hits: player then opponent
        if racket_hits_ball(player.racket, ball):
            hit_response(player.racket, ball)
            player.racket.start_swing()
            last_hit = "player"
        if racket_hits_ball(opponent.racket, ball):
            hit_response(opponent.racket, ball)
            opponent.racket.start_swing()
            last_hit = "opponent"

        # net collision logic: if ball is low around net and crosses center, treat as net hit
        if abs(ball.y - NET_Y) < ball.screen_radius() + NET_HEIGHT and TABLE_MARGIN < ball.x < SCREEN_W - TABLE_MARGIN:
            # if ball has small z and is crossing, treat as net
            if ball.z < 6:
                # the side that last hit loses point
                if last_hit == "player":
                    opponent_score += 1
                    paused = True
                    rally = False
                    serve_from_top = True
                    serve_ball(ball, serve_from_top)
                    last_hit = None
                elif last_hit == "opponent":
                    player_score += 1
                    paused = True
                    rally = False
                    serve_from_top = False
                    serve_ball(ball, serve_from_top)
                    last_hit = None

        # out of bounds (top/bottom)
        if ball.y - ball.z*0.15 <= 0:
            # player scores (ball passed opponent)
            player_score += 1
            paused = True
            rally = False
            serve_from_top = True
            serve_ball(ball, serve_from_top)
            last_hit = None
        elif ball.y + ball.z*0.15 >= SCREEN_H:
            opponent_score += 1
            paused = True
            rally = False
            serve_from_top = False
            serve_ball(ball, serve_from_top)
            last_hit = None

    # DRAW
    draw_table(screen)
    draw_net(screen)

    # layering: draw smaller y first
    items = [("opponent", opponent.y, opponent), ("player", player.y, player), ("ball", ball.y - ball.z*0.15, ball)]
    items_sorted = sorted(items, key=lambda x: x[1])
    for name, _, obj in items_sorted:
        if name == "opponent":
            obj.draw_body(screen)
            obj.racket.draw(screen)
        elif name == "player":
            obj.draw_body(screen)
            obj.racket.draw(screen)
        elif name == "ball":
            obj.draw(screen)

    # HUD
    score_surf = font_large.render(f"{opponent_score}   —   {player_score}", True, SCORE_COLOR)
    screen.blit(score_surf, (SCREEN_W//2 - score_surf.get_width()//2, 18))
    hint = "A/D or ←/→ move  •  Q/E rotate  •  SPACE serve/hit  •  R reset"
    screen.blit(font_small.render(hint, True, (230,230,230)), (20, SCREEN_H - 30))

    if paused:
        ptxt = "PAUSED — Press SPACE to serve" if not rally else "PAUSED"
        screen.blit(font_small.render(ptxt, True, (250,250,250)), (SCREEN_W//2 - 100, SCREEN_H//2 - 10))

    # check match win
    if (player_score >= MAX_SCORE or opponent_score >= MAX_SCORE) and abs(player_score - opponent_score) >= WIN_BY:
        winner = "YOU WIN!" if player_score > opponent_score else "YOU LOSE"
        big = font_large.render(winner + "  (R to restart)", True, (200,255,200))
        screen.blit(big, (SCREEN_W//2 - big.get_width()//2, SCREEN_H//2 + 30))

    pygame.display.flip()

pygame.quit()


In [2]:
"""
Accumulator: Operator Shooter
A math-driven arcade puzzle in Pygame.

Controls:
  ← / →   : move shooter
  1 or Q  : select '+'
  2 or W  : select '-'
  3 or E  : select '*'
  4 or R  : select '/'
  Space   : fire selected operator
  Esc     : quit

Objective:
  Transform your accumulator into the target value by shooting operators
  at falling numbered tiles. Each hit applies accumulator = accumulator <op> tile_value.

Author: ChatGPT
"""

import pygame, random, math, sys, time

# ---------- CONFIG ----------
WIDTH, HEIGHT = 900, 700
FPS = 60

SPAWN_INTERVAL = 1.0        # seconds between new falling numbers (will decrease with level)
FALL_SPEED_BASE = 80        # px/sec base
PLAYER_SPEED = 280          # px/sec
BULLET_SPEED = 420          # px/sec
TIME_LIMIT = 60             # seconds to solve
MAX_MISSES = 8              # allowed misses before game over
TARGET_MIN, TARGET_MAX = 1, 200

FONT_NAME = "dejavusans"

# ---------- PYGAME SETUP ----------
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Accumulator: Operator Shooter")
clock = pygame.time.Clock()
font_big = pygame.font.SysFont(FONT_NAME, 36, bold=True)
font_med = pygame.font.SysFont(FONT_NAME, 24)
font_small = pygame.font.SysFont(FONT_NAME, 18)

# ---------- GAME OBJECTS ----------
class FallingNumber:
    def __init__(self, value, x, y, speed):
        self.value = value
        self.x = x
        self.y = y
        self.speed = speed
        self.radius = 26
        self.hit = False
    def update(self, dt):
        self.y += self.speed * dt
    def draw(self, surf):
        pygame.draw.circle(surf, (240, 240, 210), (int(self.x), int(self.y)), self.radius)
        txt = font_med.render(str(self.value), True, (30,30,30))
        rect = txt.get_rect(center=(int(self.x), int(self.y)))
        surf.blit(txt, rect)

class Bullet:
    def __init__(self, op, x, y):
        self.op = op            # one of '+','-','*','/'
        self.x = x
        self.y = y
        self.speed = -BULLET_SPEED
        self.radius = 12
    def update(self, dt):
        self.y += self.speed * dt
    def draw(self, surf):
        color = (160,220,255) if self.op == '+' else (255,195,195) if self.op == '-' else (200,255,200) if self.op == '*' else (255,240,160)
        pygame.draw.circle(surf, color, (int(self.x), int(self.y)), self.radius)
        txt = font_small.render(self.op, True, (30,30,30))
        rect = txt.get_rect(center=(int(self.x), int(self.y)))
        surf.blit(txt, rect)

# ---------- GAME STATE ----------
def new_target(level):
    # target range can scale by level
    return random.randint(max(1, TARGET_MIN + level*2), TARGET_MAX + level*5)

def spawn_number(level):
    # spawn value biased by level
    base = 1 + level
    if random.random() < 0.12:
        v = random.randint(10 + level*2, 40 + level*6)
    else:
        v = random.randint(1, min(20 + level*3, 100))
    x = random.randint(60, WIDTH - 60)
    speed = FALL_SPEED_BASE + random.uniform(0, 40) + level*6
    return FallingNumber(v, x, -40, speed)

def apply_op(acc, op, val):
    try:
        if op == '+':
            return acc + val
        if op == '-':
            return acc - val
        if op == '*':
            return acc * val
        if op == '/':
            # avoid division by zero; if val==0, treat as no-op
            if val == 0:
                return acc
            return acc / val
    except Exception:
        return acc

def approx_equal(a, b, tol=1e-6):
    return abs(a - b) <= tol

# Initialize
level = 0
accumulator = random.randint(1,9)
target = new_target(level)
numbers = []
bullets = []
last_spawn_t = 0.0
start_time = time.time()
misses = 0
score = 0
selected_op = '+'
game_over = False
win = False

# ---------- UI Helpers ----------
def draw_hud():
    # top info panel
    pygame.draw.rect(screen, (18,18,18), (0,0, WIDTH, 72))
    # target
    ttxt = font_big.render(f"TARGET: {str(target)}", True, (255,230,160))
    screen.blit(ttxt, (18,12))
    # accumulator
    atxt = font_big.render(f"ACC: {format_number(accumulator)}", True, (180,255,200))
    screen.blit(atxt, (WIDTH//2 - atxt.get_width()//2,12))
    # selected op
    optxt = font_big.render(f"OPERATOR: [{selected_op}]", True, (200,200,255))
    screen.blit(optxt, (WIDTH - optxt.get_width() - 18, 12))
    # time left
    elapsed = time.time() - start_time
    rem = max(0, TIME_LIMIT - elapsed)
    timtxt = font_med.render(f"TIME: {int(rem)}s", True, (220,220,220))
    screen.blit(timtxt, (18, 44))
    # misses and score
    mstxt = font_med.render(f"MISSES: {misses}/{MAX_MISSES}", True, (255,180,180))
    screen.blit(mstxt, (WIDTH - 220, 44))
    sctxt = font_med.render(f"SCORE: {score}", True, (200,255,200))
    screen.blit(sctxt, (WIDTH//2 - 50, 44))

def format_number(x):
    # show integers without .0
    if isinstance(x, float):
        if abs(x - round(x)) < 1e-9:
            return str(int(round(x)))
        else:
            # cap decimal length
            return f"{x:.4g}"
    return str(x)

# ---------- MAIN LOOP ----------
player_x = WIDTH // 2
player_y = HEIGHT - 36
player_w = 120
player_h = 24

running = True
while running:
    dt = clock.tick(FPS) / 1000.0
    now = time.time()

    # input
    for ev in pygame.event.get():
        if ev.type == pygame.QUIT:
            running = False
        elif ev.type == pygame.KEYDOWN:
            if ev.key == pygame.K_ESCAPE:
                running = False
            if game_over:
                if ev.key == pygame.K_RETURN:
                    # restart
                    level = 0
                    accumulator = random.randint(1,9)
                    target = new_target(level)
                    numbers.clear()
                    bullets.clear()
                    last_spawn_t = now
                    start_time = time.time()
                    misses = 0
                    score = 0
                    selected_op = '+'
                    game_over = False
                    win = False
            else:
                if ev.key in (pygame.K_1, pygame.K_q):
                    selected_op = '+'
                elif ev.key in (pygame.K_2, pygame.K_w):
                    selected_op = '-'
                elif ev.key in (pygame.K_3, pygame.K_e):
                    selected_op = '*'
                elif ev.key in (pygame.K_4, pygame.K_r):
                    selected_op = '/'
                elif ev.key == pygame.K_SPACE:
                    # fire a bullet
                    bullets.append(Bullet(selected_op, player_x, player_y - 20))

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        player_x -= PLAYER_SPEED * dt
    if keys[pygame.K_RIGHT]:
        player_x += PLAYER_SPEED * dt
    player_x = max(40, min(WIDTH - 40, player_x))

    # spawn numbers depending on level and time
    if not game_over and now - last_spawn_t > max(0.25, SPAWN_INTERVAL - level*0.05):
        numbers.append(spawn_number(level))
        last_spawn_t = now

    # update falling numbers
    for n in numbers:
        n.update(dt)
    # update bullets
    for b in bullets:
        b.update(dt)

    # collision: bullets vs numbers
    remove_nums = []
    remove_bullets = []
    for i, b in enumerate(bullets):
        for j, n in enumerate(numbers):
            # circle collision
            dx = b.x - n.x
            dy = b.y - n.y
            dist = math.hypot(dx, dy)
            if dist < b.radius + n.radius:
                # perform operation
                old_acc = accumulator
                accumulator = apply_op(accumulator, b.op, n.value)
                # clamp very small floating errors
                if isinstance(accumulator, float) and abs(accumulator - round(accumulator)) < 1e-9:
                    accumulator = round(accumulator)
                # scoring: if operation moved closer to target, small score
                prev_diff = abs(old_acc - target)
                new_diff = abs(accumulator - target)
                if new_diff < prev_diff:
                    score += 1
                else:
                    score -= 0 if score <= 0 else 0  # don't harshly penalize
                remove_nums.append(n)
                remove_bullets.append(b)
                break

    # remove collided
    for n in remove_nums:
        if n in numbers:
            numbers.remove(n)
    for b in remove_bullets:
        if b in bullets:
            bullets.remove(b)

    # remove bullets that go off-screen
    bullets = [b for b in bullets if b.y + b.radius > -50]

    # check numbers reached bottom (misses)
    for n in list(numbers):
        if n.y - n.radius > HEIGHT:
            numbers.remove(n)
            misses += 1
            # small penalty to accumulator to add challenge (optional)
            # accumulator = accumulator * 0.98 if isinstance(accumulator, float) else accumulator

    # check win/lose
    if not game_over:
        # check approximate equality
        if approx_equal(accumulator, target, tol=1e-6):
            win = True
            game_over = True
        # time up
        if time.time() - start_time > TIME_LIMIT:
            game_over = True
            win = approx_equal(accumulator, target, tol=1e-6)
        # too many misses
        if misses >= MAX_MISSES:
            game_over = True
            win = False

    # background
    screen.fill((24, 24, 30))
    # draw playfield
    pygame.draw.rect(screen, (18, 90, 50), (40, 88, WIDTH - 80, HEIGHT - 160), border_radius=8)
    # draw midline
    pygame.draw.line(screen, (200,200,200), (40, HEIGHT//2), (WIDTH-40, HEIGHT//2), 1)

    # draw items
    for n in numbers:
        n.draw(screen)
    for b in bullets:
        b.draw(screen)

    # draw player
    pygame.draw.rect(screen, (200,200,255), (player_x - player_w//2, player_y - player_h//2, player_w, player_h), border_radius=10)
    ptxt = font_small.render("SHOOTER", True, (20,20,40))
    screen.blit(ptxt, (player_x - ptxt.get_width()//2, player_y - 34))

    # HUD
    draw_hud()

    # help
    help_txt = font_small.render("Pick operator (1/2/3/4 or Q/W/E/R). Space to shoot. Reach the TARGET.", True, (230,230,230))
    screen.blit(help_txt, (40, HEIGHT - 48))

    # game over panel
    if game_over:
        s = "YOU WIN!" if win else "GAME OVER"
        mid = font_big.render(s, True, (255,240,180))
        sub = font_med.render("Press Enter to play again", True, (220,220,220))
        screen.blit(mid, (WIDTH//2 - mid.get_width()//2, HEIGHT//2 - 40))
        screen.blit(sub, (WIDTH//2 - sub.get_width()//2, HEIGHT//2 + 6))

    pygame.display.flip()

pygame.quit()
sys.exit()


SystemExit: 

In [4]:
"""
Accumulator: Operator Shooter
A math-driven arcade puzzle in Pygame.

Controls:
  ← / →   : move shooter
  1 or Q  : select '+'
  2 or W  : select '-'
  3 or E  : select '*'
  4 or R  : select '/'
  Space   : fire selected operator
  Esc     : quit

Objective:
  Transform your accumulator into the target value by shooting operators
  at falling numbered tiles. Each hit applies accumulator = accumulator <op> tile_value.

Author: ChatGPT
"""

import pygame, random, math, sys, time

# ---------- CONFIG ----------
WIDTH, HEIGHT = 900, 700
FPS = 60

SPAWN_INTERVAL = 1.0        # seconds between new falling numbers (will decrease with level)
FALL_SPEED_BASE = 80        # px/sec base
PLAYER_SPEED = 280          # px/sec
BULLET_SPEED = 420          # px/sec
TIME_LIMIT = 60             # seconds to solve
MAX_MISSES = 8              # allowed misses before game over
TARGET_MIN, TARGET_MAX = 1, 200

FONT_NAME = "dejavusans"

# ---------- PYGAME SETUP ----------
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Accumulator: Operator Shooter")
clock = pygame.time.Clock()
font_big = pygame.font.SysFont(FONT_NAME, 36, bold=True)
font_med = pygame.font.SysFont(FONT_NAME, 24)
font_small = pygame.font.SysFont(FONT_NAME, 18)

# ---------- GAME OBJECTS ----------
class FallingNumber:
    def __init__(self, value, x, y, speed):
        self.value = value
        self.x = x
        self.y = y
        self.speed = speed
        self.radius = 26
        self.hit = False
    def update(self, dt):
        self.y += self.speed * dt
    def draw(self, surf):
        pygame.draw.circle(surf, (240, 240, 210), (int(self.x), int(self.y)), self.radius)
        txt = font_med.render(str(self.value), True, (30,30,30))
        rect = txt.get_rect(center=(int(self.x), int(self.y)))
        surf.blit(txt, rect)

class Bullet:
    def __init__(self, op, x, y):
        self.op = op            # one of '+','-','*','/'
        self.x = x
        self.y = y
        self.speed = -BULLET_SPEED
        self.radius = 12
    def update(self, dt):
        self.y += self.speed * dt
    def draw(self, surf):
        color = (160,220,255) if self.op == '+' else (255,195,195) if self.op == '-' else (200,255,200) if self.op == '*' else (255,240,160)
        pygame.draw.circle(surf, color, (int(self.x), int(self.y)), self.radius)
        txt = font_small.render(self.op, True, (30,30,30))
        rect = txt.get_rect(center=(int(self.x), int(self.y)))
        surf.blit(txt, rect)

# ---------- GAME STATE ----------
def new_target(level):
    # target range can scale by level
    return random.randint(max(1, TARGET_MIN + level*2), TARGET_MAX + level*5)

def spawn_number(level):
    # spawn value biased by level
    base = 1 + level
    if random.random() < 0.12:
        v = random.randint(10 + level*2, 40 + level*6)
    else:
        v = random.randint(1, min(20 + level*3, 100))
    x = random.randint(60, WIDTH - 60)
    speed = FALL_SPEED_BASE + random.uniform(0, 40) + level*6
    return FallingNumber(v, x, -40, speed)

def apply_op(acc, op, val):
    try:
        if op == '+':
            return acc + val
        if op == '-':
            return acc - val
        if op == '*':
            return acc * val
        if op == '/':
            # avoid division by zero; if val==0, treat as no-op
            if val == 0:
                return acc
            return acc / val
    except Exception:
        return acc

def approx_equal(a, b, tol=1e-6):
    return abs(a - b) <= tol

# Initialize
level = 0
accumulator = random.randint(1,9)
target = new_target(level)
numbers = []
bullets = []
last_spawn_t = 0.0
start_time = time.time()
misses = 0
score = 0
selected_op = '+'
game_over = False
win = False

# ---------- UI Helpers ----------
def draw_hud():
    # top info panel
    pygame.draw.rect(screen, (18,18,18), (0,0, WIDTH, 72))
    # target
    ttxt = font_big.render(f"TARGET: {str(target)}", True, (255,230,160))
    screen.blit(ttxt, (18,12))
    # accumulator
    atxt = font_big.render(f"ACC: {format_number(accumulator)}", True, (180,255,200))
    screen.blit(atxt, (WIDTH//2 - atxt.get_width()//2,12))
    # selected op
    optxt = font_big.render(f"OPERATOR: [{selected_op}]", True, (200,200,255))
    screen.blit(optxt, (WIDTH - optxt.get_width() - 18, 12))
    # time left
    elapsed = time.time() - start_time
    rem = max(0, TIME_LIMIT - elapsed)
    timtxt = font_med.render(f"TIME: {int(rem)}s", True, (220,220,220))
    screen.blit(timtxt, (18, 44))
    # misses and score
    mstxt = font_med.render(f"MISSES: {misses}/{MAX_MISSES}", True, (255,180,180))
    screen.blit(mstxt, (WIDTH - 220, 44))
    sctxt = font_med.render(f"SCORE: {score}", True, (200,255,200))
    screen.blit(sctxt, (WIDTH//2 - 50, 44))

def format_number(x):
    # show integers without .0
    if isinstance(x, float):
        if abs(x - round(x)) < 1e-9:
            return str(int(round(x)))
        else:
            # cap decimal length
            return f"{x:.4g}"
    return str(x)

# ---------- MAIN LOOP ----------
player_x = WIDTH // 2
player_y = HEIGHT - 36
player_w = 120
player_h = 24

running = True
while running:
    dt = clock.tick(FPS) / 1000.0
    now = time.time()

    # input
    for ev in pygame.event.get():
        if ev.type == pygame.QUIT:
            running = False
        elif ev.type == pygame.KEYDOWN:
            if ev.key == pygame.K_ESCAPE:
                running = False
            if game_over:
                if ev.key == pygame.K_RETURN:
                    # restart
                    level = 0
                    accumulator = random.randint(1,9)
                    target = new_target(level)
                    numbers.clear()
                    bullets.clear()
                    last_spawn_t = now
                    start_time = time.time()
                    misses = 0
                    score = 0
                    selected_op = '+'
                    game_over = False
                    win = False
            else:
                if ev.key in (pygame.K_1, pygame.K_q):
                    selected_op = '+'
                elif ev.key in (pygame.K_2, pygame.K_w):
                    selected_op = '-'
                elif ev.key in (pygame.K_3, pygame.K_e):
                    selected_op = '*'
                elif ev.key in (pygame.K_4, pygame.K_r):
                    selected_op = '/'
                elif ev.key == pygame.K_SPACE:
                    # fire a bullet
                    bullets.append(Bullet(selected_op, player_x, player_y - 20))

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        player_x -= PLAYER_SPEED * dt
    if keys[pygame.K_RIGHT]:
        player_x += PLAYER_SPEED * dt
    player_x = max(40, min(WIDTH - 40, player_x))

    # spawn numbers depending on level and time
    if not game_over and now - last_spawn_t > max(0.25, SPAWN_INTERVAL - level*0.05):
        numbers.append(spawn_number(level))
        last_spawn_t = now

    # update falling numbers
    for n in numbers:
        n.update(dt)
    # update bullets
    for b in bullets:
        b.update(dt)

    # collision: bullets vs numbers
    remove_nums = []
    remove_bullets = []
    for i, b in enumerate(bullets):
        for j, n in enumerate(numbers):
            # circle collision
            dx = b.x - n.x
            dy = b.y - n.y
            dist = math.hypot(dx, dy)
            if dist < b.radius + n.radius:
                # perform operation
                old_acc = accumulator
                accumulator = apply_op(accumulator, b.op, n.value)
                # clamp very small floating errors
                if isinstance(accumulator, float) and abs(accumulator - round(accumulator)) < 1e-9:
                    accumulator = round(accumulator)

                # ---------- SCORING ----------
                score += 1  # every hit gives +1
                if approx_equal(accumulator, target, tol=1e-6):
                    score = max(0, score - 3)  # penalty if hit reaches target

                remove_nums.append(n)
                remove_bullets.append(b)
                break

    # remove collided
    for n in remove_nums:
        if n in numbers:
            numbers.remove(n)
    for b in remove_bullets:
        if b in bullets:
            bullets.remove(b)

    # remove bullets that go off-screen
    bullets = [b for b in bullets if b.y + b.radius > -50]

    # check numbers reached bottom (misses)
    for n in list(numbers):
        if n.y - n.radius > HEIGHT:
            numbers.remove(n)
            misses += 1

    # check win/lose
    if not game_over:
        if approx_equal(accumulator, target, tol=1e-6):
            win = True
            game_over = True
        if time.time() - start_time > TIME_LIMIT:
            game_over = True
            win = approx_equal(accumulator, target, tol=1e-6)
        if misses >= MAX_MISSES:
            game_over = True
            win = False

    # background
    screen.fill((24, 24, 30))
    # draw playfield
    pygame.draw.rect(screen, (18, 90, 50), (40, 88, WIDTH - 80, HEIGHT - 160), border_radius=8)
    # draw midline
    pygame.draw.line(screen, (200,200,200), (40, HEIGHT//2), (WIDTH-40, HEIGHT//2), 1)

    # draw items
    for n in numbers:
        n.draw(screen)
    for b in bullets:
        b.draw(screen)

    # draw player
    pygame.draw.rect(screen, (200,200,255), (player_x - player_w//2, player_y - player_h//2, player_w, player_h), border_radius=10)
    ptxt = font_small.render("SHOOTER", True, (20,20,40))
    screen.blit(ptxt, (player_x - ptxt.get_width()//2, player_y - 34))

    # HUD
    draw_hud()

    # help
    help_txt = font_small.render("Pick operator (1/2/3/4 or Q/W/E/R). Space to shoot. Reach the TARGET.", True, (230,230,230))
    screen.blit(help_txt, (40, HEIGHT - 48))

    # game over panel
    if game_over:
        s = "YOU WIN!" if win else "GAME OVER"
        mid = font_big.render(s, True, (255,240,180))
        sub = font_med.render("Press Enter to play again", True, (220,220,220))
        screen.blit(mid, (WIDTH//2 - mid.get_width()//2, HEIGHT//2 - 40))
        screen.blit(sub, (WIDTH//2 - sub.get_width()//2, HEIGHT//2 + 6))

    pygame.display.flip()

pygame.quit()
sys.exit()


SystemExit: 